<a href="https://colab.research.google.com/github/diegohugo570/backup-codigos/blob/main/Curso_de_RAG_DascIA_Academy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R.A.G. (Retrieval-Augmented Generation)

**Este material foi preparado por [Anwar Hermuche](https://instagram.com/anwar.hermuche) para os alunos da [Formação em IA da DascIA](https://dascia.academy).**

Ele contém todas as explicações e aplicações que você precisa saber sobre R.A.G. para utilizar da melhor maneira possível. Sendo esse módulo 100% baseado na documentação oficial do Langchain, o maior framework de IA do mundo.

Dito isso, é um prazer estar contruindo esse material para vocês e bora pra cima.

In [ ]:
# Instalando as bibliotecas necessárias
!pip install langchain langchain-core langchain-community langchain-openai langchain-chroma beautifulsoup4 tiktoken pdfplumber faiss-cpu chromadb lark --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.5 MB/s eta 0:00:00
 

In [ ]:
# Configurando a rastreabilidade da aplicação
import os # operational system
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Introdução ao RAG

Antes de falarmos sobre R.A.G. especificamente, é importantíssimo lembrarmos de uma coisa - ou vê-la em um primeiro momento caso você ainda não saiba: **95% dos dados do mundo são "privados"**, ou seja, pertencentes a empresas.

Apesar disso, podemos fornecer esses dados aos LLMs! Ou seja, se você possui, na sua empresa ou na empresa que você está atendendo, um relatório privado ou dados internos pertencentes a alguma planilha e quer construir uma aplicação em cima desses documentos, você precisa, de alguma maneira, fornecer esses dados ao modelo.

E essa é uma necessidade básica que temos, até mesmo por conta das alucinações. Observe a imagem abaixo:

<img src="https://i.ibb.co/VYpfbM6X/Usua-rios-similares-37.png">

## LLMs como SOs?

Veja que um LLM hoje consegue ter acesso a entrada e saída de dados (I/O), conexão com a internet, executar código e muito mais. É realmente como se o LLM fosse o sistema operacional de um computador, a parte que controla todo o funcionamento do computador.

No canto inferior esquerdo, note o "File system", ou seja, os arquivos do sistema, que na analogia é o disco do computador, seu HD, sua memória. Os arquivos que ele tem acesso. E é justamente esse ponto que iremos trabalhar no R.A.G.. De tudo o que o LLM tem acesso, vamos focar um módulo inteiro nesse "disco".

## A problemática

E como você vai fazer isso? Vamos supor que você tem um documento de 1500 páginas para responder perguntas sobre ele. Como você vai fazer? Vai fornecer tudo de uma vez só para o modelo? Bem... **não é assim que funciona**. Já vimos sobre janela de contexto e ele não iria suportar.

Observe o código abaixo (que, por sinal, vamos aprender a como construir isso do zero):


In [ ]:
# Bibliotecas necessárias
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

## INDEXAÇÃO ##

# Carregando os documentos
loader = PDFPlumberLoader(
    file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
)

docs = loader.load()

# Pegando todo o conteúdo do livro
conteudo = "\n".join([doc.page_content for doc in docs])

In [ ]:
## GERAÇÃO ##

# Fazendo uma pergunta ao documento
llm = ChatOpenAI(model = "gpt-4o", temperature = 0)
template = f"""
Com base no contexto, responda a pergunta entre crases triplas. Se não souber, diga que não sabe.

Contexto:
{conteudo}
"""
prompt = PromptTemplate.from_template(template + """

Pergunta:
```{pergunta}```""")

rag = (
    {"pergunta": RunnablePassthrough()} |
    prompt |
    llm |
    StrOutputParser()
)

resposta = rag.invoke("O que é motor de crescimento?")

print(resposta)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 129371 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

## Possível solução?
E se pudéssemos usar apenas os chunks - que chamaremos de documentos ou chunks - que respondem as perguntas que fazemos?

Ou seja, extrair do livro apenas os chunks abaixo (que respondem a pergunta "**O que é motor de crescimento?**")

<hr>

Documento 0:

motores de crescimento. Cada um é como um motor de combustão, girando repetidas vezes.
Quanto mais rápido o ciclo é completado, mais rápido a empresa crescerá. Cada motor possui
um conjunto intrínseco de métricas que determinam com que rapidez uma empresa pode
crescer ao utilizá-lo.
OS TRÊS MOTORES DE CRESCIMENTO
Vimos na Parte II como é importante que as startups utilizem o tipo certo de métricas –
métricas acionáveis – para avaliar o progresso. No entanto, isso deixa uma grande quantidade
em termos de que números devemos medir. De fato, uma das formas mais onerosas de possível
<hr>

Documento 1:

DE ONDE VEM O CRESCIMENTO?
O motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento
sustentável. Utilizo a palavra sustentável para excluir todas as atividades ocasionais que
geram um surto de clientes, mas não têm impacto a longo prazo, tais como anúncios isolados
ou uma proeza publicitária que pode ser utilizada para revitalizar o crescimento, mas não
consegue sustentá-lo a longo prazo.
O crescimento sustentável se caracteriza por uma regra simples:
Os novos clientes surgem das ações dos clientes passados.
<hr>

Documento 2:

todos os motores de crescimento acabam ficando sem gasolina. Todos os motores estão
relacionados a um determinado conjunto de clientes e seus hábitos, preferências, canais
publicitários e interconexões. Em algum momento, esse conjunto de clientes será exaurido.
Pode levar muito ou pouco tempo, dependendo do setor e do timing.
O Capítulo 6 enfatizou a importância de construir o produto mínimo viável de maneira a não
incluir nenhum recurso adicional além do requerido pelos adotantes iniciais. Seguir essa
estratégia com êxito destravará um motor de crescimento que pode alcançar a audiência-alvo.

<hr>

Quando jogamos isso no prompt a IA consegue responder a pergunta sem dar o erro de limite de contexto, observe abaixo:

In [ ]:
# Pergunta e documentos que usaremos de contexto
pergunta = "O que é motor de crescimento?"

docs = """
Documento 0:
motores de crescimento. Cada um é como um motor de combustão, girando repetidas vezes.
Quanto mais rápido o ciclo é completado, mais rápido a empresa crescerá. Cada motor possui
um conjunto intrínseco de métricas que determinam com que rapidez uma empresa pode
crescer ao utilizá-lo.
OS TRÊS MOTORES DE CRESCIMENTO
Vimos na Parte II como é importante que as startups utilizem o tipo certo de métricas –
métricas acionáveis – para avaliar o progresso. No entanto, isso deixa uma grande quantidade
em termos de que números devemos medir. De fato, uma das formas mais onerosas de possível

Documento 1:
DE ONDE VEM O CRESCIMENTO?
O motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento
sustentável. Utilizo a palavra sustentável para excluir todas as atividades ocasionais que
geram um surto de clientes, mas não têm impacto a longo prazo, tais como anúncios isolados
ou uma proeza publicitária que pode ser utilizada para revitalizar o crescimento, mas não
consegue sustentá-lo a longo prazo.
O crescimento sustentável se caracteriza por uma regra simples:
Os novos clientes surgem das ações dos clientes passados.

Documento 2:
todos os motores de crescimento acabam ficando sem gasolina. Todos os motores estão
relacionados a um determinado conjunto de clientes e seus hábitos, preferências, canais
publicitários e interconexões. Em algum momento, esse conjunto de clientes será exaurido.
Pode levar muito ou pouco tempo, dependendo do setor e do timing.
O Capítulo 6 enfatizou a importância de construir o produto mínimo viável de maneira a não
incluir nenhum recurso adicional além do requerido pelos adotantes iniciais. Seguir essa
estratégia com êxito destravará um motor de crescimento que pode alcançar a audiência-alvo.
"""

prompt = f"""
Com base no contexto, responda a pergunta entre crases triplas. Se não souber, diga que não sabe.

Pergunta:
```{pergunta}```

Contexto:
{docs}
"""

resposta = llm.invoke(prompt)

print(resposta.content)

O motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento sustentável. Ele se caracteriza por um processo em que os novos clientes surgem das ações dos clientes passados, excluindo atividades ocasionais que geram apenas surtos temporários de clientes sem impacto a longo prazo.


Notou que usamos apenas os chunks que respondem a pergunta do usuário? **É justamente esse o propósito do RAG**.

Ao invés, de colocar todo o livro no modelo para usar de contexto, vamos **colocar apenas o chunk que responde à pergunta feita**? Faz mais sentido, não? E outra, não teria o problema da janela de contexto.

Nu! Se sua cabeça não explodiu agora, volte e leia de novo, porque era pra ter explodido. Na imagem abaixo você consegue ver a arquitetura do R.A.G.

Para facilitar a explicação, vou dividir esse fluxo em três etapas: Indexação, Recuperação e Geração. Observe abaixo:

<img src="https://i.ibb.co/39hZh10z/Lead-Entra-na-Lista-2.png">

A parte que faltou no exemplo anterior foi um pedaço da indexação e a recuperação inteira. Vamos refazer o exemplo anterior finalizando o projeto.

In [ ]:
# Bibliotecas necessárias
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_chroma.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from uuid import uuid4

## INDEXAÇÃO ##

# Carregando os documentos
loader = PDFPlumberLoader(
    file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
)

docs = loader.load()

# Dividindo o documento em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

# Embedding
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Vector Store
vector_store = Chroma(
    collection_name="eric_ries_lean_startup",
    embedding_function=embeddings,
    persist_directory="/content/chroma_langchain_db",
)

uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

['47c224ce-e0c8-4537-a8fd-9f155b68f6a3',
 '7d1bc5ac-db64-4540-b2b8-3fcfbbb84c7e',
 '495be8a3-6f50-4616-baf8-1167dfec6d4a',
 'ab89ee91-8a62-47ef-99bb-55d52e164dfd',
 '393ce2a3-0467-49ca-a8b2-d3be5787aeca',
 'd8fd6a19-bc22-4835-8dc5-8ebd1b1b3835',
 'e1e1399f-2181-4d64-8432-fd993229de3f',
 'debcda9a-ddfb-40e4-85ce-5064b79ad5f5',
 'd9cdf6ab-17fe-4816-89c0-582b38a33573',
 '40132de8-c7c9-43bc-8503-e7fd4b5c0539',
 '0c3e7366-fe35-489c-ba8e-20d4740a19ca',
 'b28f132e-c04e-46dd-a0de-9392fadacd95',
 '0cd52469-d1c7-4121-9713-d8e743dc332e',
 '288cc4e6-870c-41b2-96f7-c994d034ae1f',
 'ed2e70ab-f408-49e6-bd93-d500654fc411',
 '4ecbd5e0-790e-4251-86be-550a5bb6fd99',
 'e4b0045b-f107-4505-a267-0dec0f33d3a6',
 'c1d9811b-c760-435e-abb0-8f66f1425220',
 '38036dae-a240-4efd-b24c-8741eceb3f32',
 'e36dede9-ff5a-4ec2-a04c-ecef97b3420f',
 'ee7ae837-1648-4edd-a71a-40c522fe638c',
 '7e3b1485-1a1f-4e53-a4df-ee203ab47e80',
 '79244b9e-309a-48f2-ba34-c6c1c06a9df1',
 'e27fa678-836c-4602-a234-5605189fd21c',
 '02950803-d98d-

In [ ]:
# Transformando o vector store em um objeto "pesquisável"
retriever = vector_store.as_retriever()

In [ ]:
## RECUPERAÇÃO E GERAÇÃO ##

# Função que formata os documentos retornados
def formataDocumentos(docs):
  return "\n\n".join([f"Documento {k}:\n{doc.page_content}" for k, doc in enumerate(docs)])

# Fazendo uma pergunta ao documento
llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)
prompt = PromptTemplate.from_template("""
Com base no contexto, responda a pergunta do usuário. Se não souber, diga que não sabe.

Contexto:
{conteudo}

Pergunta:
{pergunta}

Sua resposta:
""")

rag = (
    {"conteudo": retriever | formataDocumentos, "pergunta": RunnablePassthrough()} |
    prompt |
    llm |
    StrOutputParser()
)

resposta = rag.invoke("De onde vem o crescimento de uma empresa?")

print(resposta)

O crescimento de uma empresa vem de um motor de crescimento, que é um mecanismo utilizado para alcançar um crescimento sustentável. Esse crescimento sustentável se caracteriza pelo fato de que novos clientes surgem das ações dos clientes passados. Existem quatro maneiras principais pelas quais os clientes passados podem impulsionar esse crescimento:

1. **Boca a boca**: Clientes satisfeitos falam sobre o produto, atraindo novos clientes.
2. **Efeito colateral da utilização do produto**: Produtos que geram status ou são virais podem influenciar outros a comprá-los.
3. **Publicidade financiada**: A publicidade deve ser paga com a receita recorrente gerada pelos clientes, permitindo a aquisição de novos clientes de forma sustentável.
4. **Compra ou uso repetido**: Produtos projetados para serem comprados repetidamente, como assinaturas ou recompras, também contribuem para o crescimento.

Essas fontes de crescimento sustentável ajudam a movimentar ciclos de feedback que são essenciais para

In [ ]:
# Busca dos documentos
retriever.invoke("De onde vem o crescimento de uma empresa?", k = 3)

[Document(id='259d5ba6-a44d-4d38-abca-e9f0bf1690a9', metadata={'Author': 'Eric Ries', 'CreationDate': "D:20150330135456+00'00'", 'Creator': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Producer': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Title': 'A Startup Enxuta', 'file_path': '/content/A Startup Enxuta - Eric Ries.pdf', 'page': 150, 'source': '/content/A Startup Enxuta - Eric Ries.pdf', 'total_pages': 210}, page_content='DE ONDE VEM O CRESCIMENTO?\nO motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento\nsustentável. Utilizo a palavra sustentável para excluir todas as atividades ocasionais que\ngeram um surto de clientes, mas não têm impacto a longo prazo, tais como anúncios isolados\nou uma proeza publicitária que pode ser utilizada para revitalizar o crescimento, mas não\nconsegue sustentá-lo a longo prazo.\nO crescimento sustentável se caracteriza por uma regra simples:\nOs novos clientes surgem das ações dos clientes passados.\nHá quatro man

Depois de cobrirmos essa parte mais básica, você terá feito o seu primeiro sistema de R.A.G. e vamos partir para técnicas mais avançadas. Aí o trem vai ficar interessante.

Fontes:
- [LLMs como SOs](https://x.com/karpathy/status/1707437820045062561)
- [Dados privados](https://x.com/RihardJarc/status/1778082161595208124)

# Indexação
Anteriormente, tivemos uma visão geral sobre R.A.G.. Você viu conceitos de uma maneira mais ampla e viu o porquê de ele ser extremamente necessário. Agora, vamos dar um *zoom in* para ver os detalhes mais de perto.

Vamos ver - com profundidade - uma das três etapas majoritárias que disse para vocês: **indexação, recuperação e geração**. Agora, vamos nos aprofundar mais na parte da indexação.

## O que é indexação?
Indexação é um processo onde fazemos os dados externos serem "pesquisáveis". É uma transformação que fazemos nos dados afim de deixá-los prontos para serem "pesquisáveis".

## O processo macro
<img src="https://i.ibb.co/39hZh10z/Lead-Entra-na-Lista-2.png">

A imagem mostra de uma forma muito didática o processo de indexação. Você pode ver que ele começa com o carregamento dos documentos, depois faz a divisão desses documentos em pedaços menores - chamados "chunks" - e, por fim, armazena esses pedaços menores em um *vector store*.

Para ter indexação, claro que precisamos de um documento, certo? Bom, o Langchain possui mais de 160 integrações diferentes para podermos realizar o processo de R.A.G..

**A primeira etapa é justamente obter o documento, que vou chamar de *Document Loading***: seja lendo um .pdf, pegando dados de uma página web, extraindo o texto de uma foto e muito mais.

**Já a segunda etapa consiste em pegar esses documentos, como um livro inteiro, e dividir em chunks menores, que vou chamar de *Splitting***. Isso é necessário porque posteriormente vamos retornar os K chunks mais similares para utilizar de contexto no LLM, e se tivermos o documento inteiro vai dar aquele mesmo erro de *tokens exceeded*.

Essa é apenas uma das razões pelas quais precisamos fazer a divisão dos documentos em chunks menores. Essa é uma razão necessária, porque, sem ela, o nosso R.A.G. nem funciona devido ao erro de tokens. Há outros motivos que não causam erros, mas pioram o desempenho do R.A.G.. Abaixo, você pode ver alguns benefícios de fazer a divisão em chunks
- Menor taxa de alucinação
- Respostas mais precisas
- Mais especificidade dos metadados (veremos mais a frente sobre metadados)

**Por fim, a última etapa é o processo de armazenar os dados em um banco de dados vetorial para que ele seja "pesquisável"**. O fato é: essa última etapa possui um passo intermediário chamado de *Criação de Embeddings*. Transformamos nosso documento textual em uma lista de números e armazenamos essa lista de números - chamada de Embedding - no banco de dados vetorial (vetorial porque, mais tecnicamente, chamamos essa lista de números de vetor de números).





## Document Loading
O bom de utilizar o Langchain é que ele já nos fornece integração nativa com diversos aplicativos para já extrair o conteúdo dos nossos materiais em formato de *Document*.

### O formato Document
Esse fomato Document é utilizado pelo Langchain para representar todo e qualquer tipo de conteúdo que as IAs vão utilizar. Ele possui dois atributos principais:
- page_content
- metadata

#### page_content
O atributo *page_content* possui o conteúdo textual daquele chunk. É fácil confundir e pensar que trata-se do conteúdo de uma página inteira por causa do **page**_content, mas não. É o conteúdo textual de um chunk!

#### metadata
Já o atributo *metadata* possui informações específicas sobre aquele chunk específico. No exemplo de um PDF de um livro, os metadados poderiam ser o **nome do autor**, a **página**, o **total de páginas** e muito mais. Se o material utilizado fosse a transcrição de um vídeo do YouTube, um dos metadados poderia ser o ***timestamp***, isto é, o tempo de início daquela fala da transcrição.

**Sempre que possível considere adicionar metadados aos chunks! Pode ajudar futuramente.**

### Exemplo
Para exemplificar, vou utilizar um dos chunks do .pdf do livro do Eric Ries. Mas antes, precisamos criar a nossa classe Document.

In [ ]:
# Criando a classe Document
from pydantic import BaseModel

class Document(BaseModel):
  page_content: str
  metadata: dict

# Não precisamos criar a classe Document, podemos simplesmente importar
# from langchain.schema import Document

In [ ]:
# Documento de exemplo
doc = Document(metadata={'Author': 'Eric Ries', 'CreationDate': "D:20150330135456+00'00'", 'Creator': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Producer': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Title': 'A Startup Enxuta', 'file_path': '/content/A Startup Enxuta - Eric Ries.pdf', 'page': 150, 'source': '/content/A Startup Enxuta - Eric Ries.pdf', 'total_pages': 210}, page_content='DE ONDE VEM O CRESCIMENTO?\nO motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento\nsustentável. Utilizo a palavra sustentável para excluir todas as atividades ocasionais que\ngeram um surto de clientes, mas não têm impacto a longo prazo, tais como anúncios isolados\nou uma proeza publicitária que pode ser utilizada para revitalizar o crescimento, mas não\nconsegue sustentá-lo a longo prazo.\nO crescimento sustentável se caracteriza por uma regra simples:\nOs novos clientes surgem das ações dos clientes passados.\nHá quatro maneiras principais de os clientes passados impulsionarem o crescimento\nsustentável:\n1. Boca a boca. Um nível natural de crescimento está integrado na maioria dos produtos\nprovocado pelo entusiasmo dos clientes satisfeitos com o produto. Por exemplo, quando\ncomprei meu primeiro DVR TiVo, não conseguia parara de falar dele para meus amigos e\nfamiliares. Em pouco tempo, toda minha família estava usando esse aparelho.\n2. Como efeito colateral da utilização do produto. Produtos de moda ou status, tais como\nbens de luxo, promovem a consciência de si mesmos sempre que são usados. Quando você vê\nalguém vestido com uma roupa de grife ou dirigindo um certo carro, você pode ser\ninfluenciado a comprar aquele produto. Isso também é verdade em relação aos assim\nchamados produtos virais, como Facebook ou PayPal. Quando um cliente envia dinheiro para\num amigo usando o PayPal, o amigo fica exposto automaticamente ao produto PayPal.\n3. Por meio de publicidade financiada. A maioria das empresas utiliza a publicidade para\nincitar novos clientes a usar seus produtos. Para isso ser uma fonte de crescimento sustentável,\na publicidade deve ser paga como resultado da receita recorrente, não de fontes ocasionais,\ncomo o capital de investimento. Enquanto o custo de adquirir um novo cliente (o assim\nchamado custo marginal) é menor do que a receita gerada pelos clientes (a receita marginal), o\nexcesso (o lucro marginal) pode ser utilizado para conseguir mais clientes. Quanto maior o\nlucro marginal, mais rápido o crescimento.\n4. Por meio da compra ou do uso repetido. Alguns produtos são projetados para ser\ncomprados repetidas vezes por meio de um plano de assinatura (uma empresa de tevê a cabo)\nou de recompras voluntárias (comestíveis ou lâmpadas). Porém, diversos produtos e serviços\nsão deliberadamente projetados como eventos ocasionais: por exemplo, o planejamento de um\ncasamento.\nEssas fontes de crescimento sustentável movimentam os ciclos de feedback que denominei\n')

In [ ]:
doc.page_content

'DE ONDE VEM O CRESCIMENTO?\nO motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento\nsustentável. Utilizo a palavra sustentável para excluir todas as atividades ocasionais que\ngeram um surto de clientes, mas não têm impacto a longo prazo, tais como anúncios isolados\nou uma proeza publicitária que pode ser utilizada para revitalizar o crescimento, mas não\nconsegue sustentá-lo a longo prazo.\nO crescimento sustentável se caracteriza por uma regra simples:\nOs novos clientes surgem das ações dos clientes passados.\nHá quatro maneiras principais de os clientes passados impulsionarem o crescimento\nsustentável:\n1. Boca a boca. Um nível natural de crescimento está integrado na maioria dos produtos\nprovocado pelo entusiasmo dos clientes satisfeitos com o produto. Por exemplo, quando\ncomprei meu primeiro DVR TiVo, não conseguia parara de falar dele para meus amigos e\nfamiliares. Em pouco tempo, toda minha família estava usando esse aparelho.\n2. Como e

In [ ]:
doc.metadata

{'Author': 'Eric Ries',
 'CreationDate': "D:20150330135456+00'00'",
 'Creator': 'calibre 2.20.0 [http://calibre-ebook.com]',
 'Producer': 'calibre 2.20.0 [http://calibre-ebook.com]',
 'Title': 'A Startup Enxuta',
 'file_path': '/content/A Startup Enxuta - Eric Ries.pdf',
 'page': 150,
 'source': '/content/A Startup Enxuta - Eric Ries.pdf',
 'total_pages': 210}

Como podemos ver, os metadados desse chunk indicam diversas informações úteis para podermos utilizar durante o R.A.G..

### Exemplos de *loading*
Agora, vou fazer dois exemplos de integrações que o Langchain possui para fazermos o corregamento do conteúdo dos materiais. Lembrando que acima no material possui um exemplo mostrando uma integração com PDFs.

É importante dizer que a maioria deles você pode encontrar pesquisando no google: "**langchain [NOME]**", como **langchain arxiv** ou **langchain youtube**, ou **langchain pdf**. Geralmente, o primeiro link será a resposta para a sua dúvida.

#### ArXiv
O ArXiv possui diversos artifos científicos para podermos trabalhar em cima e fazer a busca no R.A.G.. A maioria das integrações específicas vai necessitar de instalação adicional de bibliotecas, como é necessário aqui no caso do ArXiv.

Observe o código abaixo realizando a integração.

In [ ]:
# Instalando o ArXiv e PyMuPDF
!pip install arxiv pymupdf --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
# Fazendo a integração
from langchain_community.document_loaders import ArxivLoader

# Instanciando o objeto de carregamento
loader = ArxivLoader(
    query = "large language models",
    load_max_docs = 3,
    load_all_available_meta = True,

)

# Carregando os materiais
docs = loader.load()

In [ ]:
# Olhando o primeiro material
doc = docs[0]
doc.page_content

'Lost in Translation\nMay 2023\nA report from\nGabriel Nicholas\nAliya Bhatia\nLarge Language Models in \nNon-English Content Analysis\nGABRIEL NICHOLAS\nResearch Fellow at the Center for Democracy & Technology.\nALIYA BHATIA\nPolicy Analyst, Free Expression Project at the Center for \nDemocracy & Technology.\nThe Center for Democracy & Technology (CDT) is the leading \nnonpartisan, nonprofit organization fighting to advance civil rights and \ncivil liberties in the digital age. We shape technology policy, governance, \nand design with a focus on equity and democratic values. Established in \n1996, CDT has been a trusted advocate for digital rights since the earliest \ndays of the internet. The organization is headquartered in Washington, \nD.C., and has a Europe Office in Brussels, Belgium.\nA report from\nGabriel Nicholas and Aliya Bhatia\nWITH CONTRIBUTIONS BY\nSamir Jain, Mallory Knodel, Emma Llansó, Michal Luria, Nathalie Maréchal, Dhanaraj Thakur, and \nCaitlin Vogus.\nACKNOWLEDG

In [ ]:
doc.metadata

{'Published': '2023-06-12',
 'Title': 'Lost in Translation: Large Language Models in Non-English Content Analysis',
 'Authors': 'Gabriel Nicholas, Aliya Bhatia',
 'Summary': "In recent years, large language models (e.g., Open AI's GPT-4, Meta's LLaMa,\nGoogle's PaLM) have become the dominant approach for building AI systems to\nanalyze and generate language online. However, the automated systems that\nincreasingly mediate our interactions online -- such as chatbots, content\nmoderation systems, and search engines -- are primarily designed for and work\nfar more effectively in English than in the world's other 7,000 languages.\nRecently, researchers and technology companies have attempted to extend the\ncapabilities of large language models into languages other than English by\nbuilding what are called multilingual language models.\n  In this paper, we explain how these multilingual language models work and\nexplore their capabilities and limits. Part I provides a simple technical\nexpl

#### YouTube
Agora que já vimos como conseguimos pegar informações de artigos científicos usando o ArxivLoader, vamos fazer o mesmo com dados do YouTube!

In [ ]:
# Instalando as bibliotecas necessárias
!pip install youtube-transcript-api youtube-search pytube --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Realizando o carregamento dos materiais
from langchain_community.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

# Termo de pesquisa, numero de vídeos retornados
links_videos = eval(tool.run("anwar hermuche,5"))

links_videos

['https://www.youtube.com/watch?v=7NLYcGVNPXQ&pp=ygUOYW53YXIgaGVybXVjaGU%3D',
 'https://www.youtube.com/watch?v=1PZesoXOL9Q&pp=ygUOYW53YXIgaGVybXVjaGU%3D',
 'https://www.youtube.com/watch?v=NgoATpWfFcw&pp=ygUOYW53YXIgaGVybXVjaGU%3D',
 'https://www.youtube.com/watch?v=YGgTfz3cNVA&pp=ygUOYW53YXIgaGVybXVjaGU%3D',
 'https://www.youtube.com/watch?v=W1QGR9SvhAw&pp=ygUOYW53YXIgaGVybXVjaGU%3D']

In [ ]:
# Pegando a transcrição em formato de documento de cada um dos vídeos
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
from time import sleep
documents = []

for link in links_videos:
  loader = YoutubeLoader.from_youtube_url(
      youtube_url = link,
      transcript_format = TranscriptFormat.CHUNKS,
      chunk_size_seconds = 90,
      add_video_info = False,
      language = ["pt"]
  )

  docs = loader.load()
  for doc in docs:
    documents.append(doc)

  sleep(2)

TypeError: 'FetchedTranscriptSnippet' object is not subscriptable

In [ ]:
# Pegando um documento de exemplo
doc = documents[210]
doc.page_content

'estiverem desenhando o funcionamento dos seus Agent workflows cara pega o scal draw pega alguma ferramenta o pint que seja e desenha bicho fala aqui ó esse cara aqui vai ser o especialista em dados especialista em dados e por aí vai e aí você fala ó esse cara inclusive ele responde op Nossa tá feio hein responde tal e tal coisa ele faz isso isso aquilo outro ele e você vai pensando nos Estados também do grafo entendeu então você vai desenhando aqui primeiro para depois implementar Essa é a melhor saída que você tem show de bola é a melhor saída que você tem então basicamente eu tô criando aqui eu tô compilando o meu grafo tá aqui ó tô criando de fato ele e aqui eu consigo visualizar o meu gráfico então você pode ver que eu consigo visualizar a partir do código que eu criei que é exatamente o gráfico que eu tô mostrando aqui para vocês só que mais bonito né então eu tive o trabalho aqui de fazer ele mais bonito mas enfim para ficar um pouco mais mais claro e e aqui ó basicamente ó Gere

In [ ]:
doc.metadata

{'source': 'https://www.youtube.com/watch?v=94F6RYxleHA&t=4680s',
 'start_seconds': 4680,
 'start_timestamp': '01:18:00'}

Aqui, tem uma pequena diferença do ArxivLoader: os Documentos já são os chunks que iremos utilizar! Perceba como temos blocos de transcrições que representam 90 segundos de vídeo.

Quando estávamos trabalhando com o ArxivLoader, estávamos recebendo o artigo completo que precisaremos, na etapa seguinte, dividir em alguns chunks menores.

## Splitting
Agora que já temos o nosso material carregado, precisamos fazer o *splitting* dele, ou seja, dividi-lo em pedaços menores de acordo com uma regra específica.

A questão aqui é que podemos fazer esse *splitting* de várias formas. Várias! E essa maneira de criação dos chunks impacta diretamente na qualidade do R.A.G., por isso vamos estudar aqui as principais técnicas.

Na maioria dos métodos de splitting, você vai encontrar dois parâmetros:
- **Chunk Size**: O número de caracteres usado em cada chunk (50, 100, 1000 etc.)
- **Chunk Overlap**: O número de caracteres que você deseja de sobreposição dos chunks sequenciais. Isso é para evitar cortar um pedaço único de contexto em diversos pedaços. Logo, esse *overlap* criará duplicatas ao longo dos chunks.

Obs.: estou usando diversos exemplos desse material feito pelo Greg Kamradt. Você pode encontrar [aqui](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb). Inclusive, [esse site](https://chunkviz.up.railway.app/) feito por ele é excelente para a visualização de chunks.

### Chracter Splitting (Divisão por Caracteres)
Esse é o método mais simples de realizar a divisão dos chunks. É o processo de dividir o seu texto em N pedaços de X caracteres cada, independentemente do contexto ou formato do texto.

<img src="https://i.ibb.co/nCbY84s/Screenshot-2024-12-28-at-18-57-55.png">

#### Exemplo

In [ ]:
# Texto de exemplo
doc = Document(page_content = "Suponha que esse seja um texto que você irá realizar o processo de splitting. É um texto realmente de exemplo!", metadata = {"autor": "Anwar Hermuche"})

In [ ]:
CHUNK_SIZE = 55 # Número de caracteres para fazer o splitting
CHUNK_OVERLAP = 25 # Número de caracteres de sobreposição

In [ ]:
# Realizando o splitting manualmente
from langchain.schema import Document
chunks = []

# Fazendo os splits
for i in range(0, len(doc.page_content), CHUNK_SIZE - CHUNK_OVERLAP):
  texto = doc.page_content
  chunk = texto[i:i+CHUNK_SIZE]
  document = Document(page_content=chunk, metadata=doc.metadata)
  document.metadata["chunk_index"] = i // (CHUNK_SIZE - CHUNK_OVERLAP) + 1
  chunks.append(document)

  if len(chunk) < CHUNK_SIZE:
    break

chunks

[Document(metadata={'autor': 'Anwar Hermuche', 'chunk_index': 1}, page_content='Suponha que esse seja um texto que você irá realizar o '),
 Document(metadata={'autor': 'Anwar Hermuche', 'chunk_index': 2}, page_content=' que você irá realizar o processo de splitting. É um te'),
 Document(metadata={'autor': 'Anwar Hermuche', 'chunk_index': 3}, page_content='sso de splitting. É um texto realmente de exemplo!')]

In [ ]:
# Utilizando langchain
from langchain.text_splitter import CharacterTextSplitter

# Instanciando o
text_splitter = CharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap = CHUNK_OVERLAP, separator = "", strip_whitespace = False)
text_splitter.split_documents([doc])

[Document(metadata={'autor': 'Anwar Hermuche'}, page_content='Suponha que esse seja um texto que você irá realizar o '),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content=' que você irá realizar o processo de splitting. É um te'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='sso de splitting. É um texto realmente de exemplo!')]

### Recursive Chracter Splitting (Divisão por Caracteres Recursiva)
Vimos no método anterior que não levamos em conta a estrutura do texto. Simplesmente contamos o número de caracteres e realizamos a divisão de acordo com algum separador. Só isso.

Aqui, utilizando o Recursive Character Text Splitter (que vou chamar de RCTS para facilitar minha vida), podemos especificar uma série de separadores que será utilizado para fazer a divisão dos chunks do nosso documento.

Por padrão, ele usa esses:
- "\n\n" - Double new line, or most commonly paragraph breaks
- "\n" - New lines
- " " - Spaces
- "" - Characters

Sempre começo minhas aplicações com ele. É um excelente ponto de partida.

#### Exemplo

In [ ]:
# Texto de exemplo
docs = [Document(page_content = "O maior enigma que eu nunca consegui desvendar sobre inteligência artificial quando comecei a estudar é como seu desenvolvimento acontece de forma acelerada.", metadata = {"autor": "Anwar Hermuche"}),
        Document(page_content = "Pesquisadores e acadêmicos costumavam afirmar com convicção que as descobertas surgiam gradualmente. 'Cada inovação demora', escutei incessantemente, 'seu tempo natural.' Tinham pensamentos honestos, mas essa visão falha. Se sua tecnologia demonstra capacidade inferior aos sistemas existentes, não atrai poucos interessados. Permanece esquecida, virando história.", metadata = {"autor": "Anwar Hermuche"}),
        Document(page_content = "Tornou-se inquestionável atualmente que o progresso da IA generativa segue curvas exponenciais na computação moderna. Céticos argumentam que essa característica prejudica o desenvolvimento sustentável, sugerindo alterações profundas nas pesquisas atuais. Porém avanços acelerados representam fenômenos naturais da evolução tecnológica, independente das metodologias estabelecidas. Observamos padrões similares em adaptabilidade, processamento, análise contextual, eficiência computacional e benefícios práticos gerados. Cada descoberta multiplica possibilidades futuras.", metadata = {"autor": "Anwar Hermuche"})]

In [ ]:
# Definindo variáveis
CHUNK_SIZE = 55
CHUNK_OVERLAP = 25

In [ ]:
# Utilizando langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Instanciando o splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap = CHUNK_OVERLAP)
text_splitter.split_documents(docs)

[Document(metadata={'autor': 'Anwar Hermuche'}, page_content='O maior enigma que eu nunca consegui desvendar sobre'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='consegui desvendar sobre inteligência artificial'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='inteligência artificial quando comecei a estudar é'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='comecei a estudar é como seu desenvolvimento acontece'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='desenvolvimento acontece de forma acelerada.'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='Pesquisadores e acadêmicos costumavam afirmar com'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content='costumavam afirmar com convicção que as descobertas'),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content="que as descobertas surgiam gradualmente. 'Cada"),
 Document(metadata={'autor': 'Anwar Hermuche'}, page_content="gradualmente. 'Ca

### Semantic Chunking (Divisão semântica)
Não sei se você se questionou isso até aqui, mas não é estranho ter um tamanho pré definido de chunks para todo o documento? Alguns chuks podem ser maiores que outros naturalmente.

Já comentamos sobre os embeddings, e eles sozinhos não fazem muita coisa. Mas quando começamos a **comparar embeddings diferentes conseguimos começar a inferir algumas relações entre os chunks**.

O que podemos fazer então? Tentar criar clusters de chunks semelhantes com o intuito de, juntos, terem um significado maior. O código abaixo você pode utilizar sem precisar entender, caso não queira.

In [ ]:
!pip install langchain_experimental --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader

# Carregando os documentos
loader = PDFPlumberLoader(
    file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
)

# Documentos
docs = loader.load()

# Splitter
text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type = "percentile")

# Documentos
docs = text_splitter.split_documents(docs)
print(docs[385].page_content)
print("-="*20)
print(docs[385].metadata)

59 Informações acerca da Alphabet Energy foram obtidas em entrevistas realizadas por Sara Leslie. 60 Para mais detalhes a respeito da organização de aprendizagem da Toyota, ver The Toyota Way (O modelo Toyota), de
Jeffrey Liker. 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
{'source': '/content/A Startup Enxuta - Eric Ries.pdf', 'file_path': '/content/A Startup Enxuta - Eric Ries.pdf', 'page': 148, 'total_pages': 210, 'Author': 'Eric Ries', 'CreationDate': "D:20150330135456+00'00'", 'Creator': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Producer': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Title': 'A Startup Enxuta'}


### Agentic Chunking (Divisão realizada por Agentes)
Já foi discutido aqui que qualquer termo chamado de agêntico faz coisas que humanos fariam. Então, vamos nos questionar: **como humanos criariam chunks**?

Bom, quando não temos chunk ainda, pegamos a primeira parte para ser um chunk. Depois disso, vemos se a segunda parte é similar o suficiente para também ser colocada no primeiro chunk. Se sim, colocamos. Se não, criamos um novo chunk e continuamos o processo.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import uuid
import os
from typing import Optional
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()

class AgenticChunker:
    def __init__(self, openai_api_key=None):
        self.chunks = {}
        self.id_truncate_limit = 5

        # Whether or not to update/refine summaries and titles as you get new information
        self.generate_new_metadata_ind = True
        self.print_logging = True

        if openai_api_key is None:
            openai_api_key = os.getenv("OPENAI_API_KEY")

        if openai_api_key is None:
            raise ValueError("API key is not provided and not found in environment variables")

        self.llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key=openai_api_key, temperature=0)

    def add_propositions(self, propositions):
        for proposition in propositions:
            self.add_proposition(proposition)

    def add_proposition(self, proposition):
        if self.print_logging:
            print (f"\nAdding: '{proposition}'")

        # If it's your first chunk, just make a new chunk and don't check for others
        if len(self.chunks) == 0:
            if self.print_logging:
                print ("No chunks, creating a new one")
            self._create_new_chunk(proposition)
            return

        chunk_id = self._find_relevant_chunk(proposition)

        # If a chunk was found then add the proposition to it
        if chunk_id:
            if self.print_logging:
                print (f"Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: {self.chunks[chunk_id]['title']}")
            self.add_proposition_to_chunk(chunk_id, proposition)
            return
        else:
            if self.print_logging:
                print ("No chunks found")
            # If a chunk wasn't found, then create a new one
            self._create_new_chunk(proposition)


    def add_proposition_to_chunk(self, chunk_id, proposition):
        # Add then
        self.chunks[chunk_id]['propositions'].append(proposition)

        # Then grab a new summary
        if self.generate_new_metadata_ind:
            self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
            self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])

    def _update_chunk_summary(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the summary or else they could get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    A new proposition was just added to one of your chunks, you should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about.

                    A good summary will say what the chunk is about, and give any clarifying instructions on what to add to the chunk.

                    You will be given a group of propositions which are in the chunk and the chunks current summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the chunk new summary, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nCurrent chunk summary:\n{current_summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary']
        }).content

        return new_chunk_summary

    def _update_chunk_title(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the title or else it can get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    A new proposition was just added to one of your chunks, you should generate a very brief updated chunk title which will inform viewers what a chunk group is about.

                    A good title will say what the chunk is about.

                    You will be given a group of propositions which are in the chunk, chunk summary and the chunk title.

                    Your title should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nChunk summary:\n{current_summary}\n\nCurrent chunk title:\n{current_title}"),
            ]
        )

        runnable = PROMPT | self.llm

        updated_chunk_title = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary'],
            "current_title" : chunk['title']
        }).content

        return updated_chunk_title

    def _get_new_chunk_summary(self, proposition):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    You should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about.

                    A good summary will say what the chunk is about, and give any clarifying instructions on what to add to the chunk.

                    You will be given a proposition which will go into a new chunk. This new chunk needs a summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the new chunk summary, nothing else.
                    """,
                ),
                ("user", "Determine the summary of the new chunk that this proposition will go into:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": proposition
        }).content

        return new_chunk_summary

    def _get_new_chunk_title(self, summary):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    You should generate a very brief few word chunk title which will inform viewers what a chunk group is about.

                    A good chunk title is brief but encompasses what the chunk is about

                    You will be given a summary of a chunk which needs a title

                    Your titles should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Determine the title of the chunk that this summary belongs to:\n{summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_title = runnable.invoke({
            "summary": summary
        }).content

        return new_chunk_title


    def _create_new_chunk(self, proposition):
        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit] # I don't want long ids
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)

        self.chunks[new_chunk_id] = {
            'chunk_id' : new_chunk_id,
            'propositions': [proposition],
            'title' : new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index' : len(self.chunks)
        }
        if self.print_logging:
            print (f"Created new chunk ({new_chunk_id}): {new_chunk_title}")

    def get_chunk_outline(self):
        """
        Get a string which represents the chunks you currently have.
        This will be empty when you first start off
        """
        chunk_outline = ""

        for chunk_id, chunk in self.chunks.items():
            single_chunk_string = f"""Chunk ID: {chunk['chunk_id']}\nChunk Name: {chunk['title']}\nChunk Summary: {chunk['summary']}\n\n"""

            chunk_outline += single_chunk_string

        return chunk_outline

    def _find_relevant_chunk(self, proposition):
        current_chunk_outline = self.get_chunk_outline()

        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    Determine whether or not the "Proposition" should belong to any of the existing chunks.

                    A proposition should belong to a chunk of their meaning, direction, or intention are similar.
                    The goal is to group similar propositions and chunks.

                    If you think a proposition should be joined with a chunk, return the chunk id.
                    If you do not think an item should be joined with an existing chunk, just return "No chunks"

                    Example:
                    Input:
                        - Proposition: "Greg really likes hamburgers"
                        - Current Chunks:
                            - Chunk ID: 2n4l3d
                            - Chunk Name: Places in San Francisco
                            - Chunk Summary: Overview of the things to do with San Francisco Places

                            - Chunk ID: 93833k
                            - Chunk Name: Food Greg likes
                            - Chunk Summary: Lists of the food and dishes that Greg likes
                    Output: 93833k
                    """,
                ),
                ("user", "Current Chunks:\n--Start of current chunks--\n{current_chunk_outline}\n--End of current chunks--"),
                ("user", "Determine if the following statement should belong to one of the chunks outlined:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        chunk_found = runnable.invoke({
            "proposition": proposition,
            "current_chunk_outline": current_chunk_outline
        }).content

        # Pydantic data class
        class ChunkID(BaseModel):
            """Extracting the chunk id"""
            chunk_id: Optional[str]

        # Extraction to catch-all LLM responses. This is a bandaid
        llm_structured = self.llm.with_structured_output(ChunkID)
        chunk_found = llm_structured.invoke(chunk_found).chunk_id

        # If you got a response that isn't the chunk id limit, chances are it's a bad response or it found nothing
        # So return nothing
        if chunk_found != None and len(chunk_found) != self.id_truncate_limit:
            return None

        return chunk_found

    def get_chunks(self, get_type='dict'):
        """
        This function returns the chunks in the format specified by the 'get_type' parameter.
        If 'get_type' is 'dict', it returns the chunks as a dictionary.
        If 'get_type' is 'list_of_strings', it returns the chunks as a list of strings, where each string is a proposition in the chunk.
        """
        if get_type == 'dict':
            return self.chunks
        if get_type == 'list_of_strings':
            chunks = []
            for chunk_id, chunk in self.chunks.items():
                chunks.append(" ".join([x for x in chunk['propositions']]))
            return chunks

    def pretty_print_chunks(self):
        print (f"\nYou have {len(self.chunks)} chunks\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"Chunk #{chunk['chunk_index']}")
            print(f"Chunk ID: {chunk_id}")
            print(f"Summary: {chunk['summary']}")
            print(f"Propositions:")
            for prop in chunk['propositions']:
                print(f"    -{prop}")
            print("\n\n")

    def pretty_print_chunk_outline(self):
        print ("Chunk Outline\n")
        print(self.get_chunk_outline())

In [ ]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Optional, List
from pydantic import BaseModel
from langchain import hub

obj = hub.pull("wfh/proposal-indexing")
llm = ChatOpenAI(model='gpt-4o-mini', temperature = 0)

print(obj.messages[0].prompt.template)

Decompose the "Content" into clear and simple propositions, ensuring they are interpretable out of
context.
1. Split compound sentence into simple sentences. Maintain the original phrasing from the input
whenever possible.
2. For any named entity that is accompanied by additional descriptive information, separate this
information into its own distinct proposition.
3. Decontextualize the proposition by adding necessary modifier to nouns or entire sentences
and replacing pronouns (e.g., "it", "he", "she", "they", "this", "that") with the full name of the
entities they refer to.
4. Present the results as a list of strings, formatted in JSON.

Example:

Input: Title: ¯Eostre. Section: Theories and interpretations, Connection to Easter Hares. Content:
The earliest evidence for the Easter Hare (Osterhase) was recorded in south-west Germany in
1678 by the professor of medicine Georg Franck von Franckenau, but it remained unknown in
other parts of Germany until the 18th century. Scholar Richar

In [ ]:
# Criando um runnable
runnable = obj | llm | StrOutputParser()

# Classe de sentença
class Sentences(BaseModel):
  sentences: List[str]

# Função para obter as proposições
def obterProposicoes(text):
  runnable_output = runnable.invoke({
    "input": text
  })

  llm_structured = llm.with_structured_output(Sentences)
  proposicoes = llm_structured.invoke(runnable_output).sentences
  return proposicoes

In [ ]:
# # Conteúdo em texto do livro
# loader = PDFPlumberLoader(
#     file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
# )
# docs = loader.load()
# conteudo = "\n\n".join([x.page_content for x in docs if x.page_content.strip() not in ("\n", "", " ", "\t")])

# O que define uma parte?
paragrafos = "Suponha que esse seja um texto que você irá realizar o processo de splitting.\n\nÉ um texto realmente de exemplo!\n\nGosto de cachorros.".split("\n\n")
len(paragrafos)

3

In [ ]:
proposicoes_livro = []

# Fazendo com apenas 5 para poder demonstrar, porque 213 é demais para uma demonstração
for i, para in enumerate(paragrafos[:5]):
  propositions = obterProposicoes(para)

  proposicoes_livro.extend(propositions)
  print(f"Feito {i+1}/5")

print("-="*20)
proposicoes_livro[:5]

Feito 1/5
Feito 2/5
Feito 3/5
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


['Suponha que esse seja um texto.',
 'Você irá realizar o processo de splitting.',
 'É um texto realmente de exemplo.',
 'A pessoa gosta de cachorros.']

In [ ]:
# Criação do agentic chunker
ac = AgenticChunker()
ac.add_propositions(proposicoes_livro)


Adding: 'Suponha que esse seja um texto.'
No chunks, creating a new one
Created new chunk (be495): Text & Written Content

Adding: 'Você irá realizar o processo de splitting.'
No chunks found
Created new chunk (696b4): Task Management

Adding: 'É um texto realmente de exemplo.'
Chunk Found (be495), adding to: Text & Written Content

Adding: 'A pessoa gosta de cachorros.'
No chunks found
Created new chunk (224d3): Pets


In [ ]:
ac.pretty_print_chunks()


You have 3 chunks

Chunk #0
Chunk ID: be495
Summary: This chunk contains example text and related written content.
Propositions:
    -Suponha que esse seja um texto.
    -É um texto realmente de exemplo.



Chunk #1
Chunk ID: 696b4
Summary: This chunk contains instructions related to the process of splitting tasks or items.
Propositions:
    -Você irá realizar o processo de splitting.



Chunk #2
Chunk ID: 224d3
Summary: This chunk contains information about the types of pets that the person likes.
Propositions:
    -A pessoa gosta de cachorros.





In [ ]:
# Pegando os chunks
chunks = ac.get_chunks(get_type='list_of_strings')

chunks

['A presente obra é disponibilizada pela equipe Le Livros e seus diversos parceiros. É totalmente repudiável o aluguel do presente conteúdo. É totalmente repudiável qualquer uso comercial do presente conteúdo.',
 'O objetivo da disponibilização da obra é oferecer conteúdo para uso parcial em pesquisas e estudos acadêmicos. O objetivo da disponibilização da obra também é permitir o simples teste da qualidade da obra. O fim exclusivo da disponibilização da obra é a compra futura.',
 'É expressamente proibida a venda do presente conteúdo.',
 'O Le Livros e seus parceiros disponibilizam conteúdo de domínio público e propriedade intelectual de forma totalmente gratuita.',
 'O Le Livros e seus parceiros acreditam que o conhecimento e a educação devem ser acessíveis e livres a toda e qualquer pessoa. Quando o mundo estiver unido na busca do conhecimento, a sociedade poderá evoluir a um novo nível.',
 'Você pode encontrar mais obras no site LeLivros.link. Você pode encontrar mais obras em qual

## Criação de Embeddings
Agora que temos os nossos dados divididos em chunks, precisamos tornar esses chunks pesquisáveis para poder retornar os chunks mais similares à query do usuário. **Mas como medimos a similaridade de dois textos?**

A primeira coisa que precisamos pensar é que realmente não tem jeito. Para mensurar a similaridade de dois textos, precisamos transforma-lo em números! E aí você pode pensar:

- "Ah, Anwar, mas tem como ser com texto. Basta contar quantas palavras iguais a query e o chunk possuem".

Concordo! É uma maneira, mas concorda comigo que podemos representar isso com números também? (se você não concorda, segura um pouco que vou te mostrar que sim, é possível)

E agora fica a questão do milênio: **qual a melhor maneira de conseguir extrair o máximo de informações de um texto quando transformamos ele em números?**

### Bag of Words
<img src="https://miro.medium.com/v2/resize:fit:1322/format:webp/0*cf1wq8eIix-Z2qIf.png">

Como disse, há várias maneiras de representar um texto numericamente para, depois, calcular a similaridade dos dois textos.

A primeira técnica que veremos é a Bag of Words, um método estatístico. Mesmo que hoje não seja tão usada assim, é importante conhecermos as técnicas para ficar mais inteligente.

Na imagem, você consegue ver como a Bag of Words funciona. **Pegamos uma frase, analisamos quais palavras únicas a frase tem e contamos quantas vezes cada palavra aparece no texto**.

Observe o exemplo abaixo:

In [ ]:
# Documentos de exemplo
doc1 = Document(page_content = "Eu amo meu cachorro! Amo!", metadata = {})
doc2 = Document(page_content = "Eu gosto do meu gato! Amo ele.", metadata = {})

Agora, vamos unir os textos de todos os documentos para construir nossa *bag of words*.

In [ ]:
# Construindo a Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

all_texts = [doc1.page_content, doc2.page_content]
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(all_texts)

print(vectorizer.get_feature_names_out())
print("-="*30)
print(bag_of_words.toarray())

['amo' 'cachorro' 'do' 'ele' 'eu' 'gato' 'gosto' 'meu']
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
[[2 1 0 0 1 0 0 1]
 [1 0 1 1 1 1 1 1]]


In [ ]:
# Vendo a similaridade das duas frases
qtd_palavras_iguais = 0
for i, palavra in enumerate(vectorizer.get_feature_names_out()):
  if bag_of_words.toarray()[0][i] != 0 and bag_of_words.toarray()[1][i] != 0:
    qtd_palavras_iguais += 1

print(f"Quantidade de palavras iguais: {qtd_palavras_iguais}")

Quantidade de palavras iguais: 3


Note que a utilização de Bag of Words faz com que a representação fique muito esparsa, e isso significa ter muitos zeros no nosso array. Imagine em um livro a quantidade de palavras únicas que ele possui. **O vocabulário pode ficar muito grande e, além disso, não estamos armazenando a informação semântica de cada documento, somente a frequência de palavras**.

Há um método de buscar documentos similares utilizando Bag of Words como representação numérica muito famoso que vamos estudar um pouco mais para frente.

### Embeddings
<img src="https://assets.zilliz.com/How_vector_embeddings_are_generated_and_stored_7e9c5a2a41.png">

Acabamos de ver um método puramente estatístico que analisa apenas a frequência de aparição das palavras em um texto. Os embeddings, um método que utiliza Machine Learning, conseguem capturar uma coisa muito mais interessante: **a semântica do texto - o seu significado**.

Concorda comigo que as frases **Estou bem cansado** e **Preciso urgentemente descansar um pouco** são frases que, semanticamente, dizem a mesma coisa, mas não possuem palavras em comum. Mesmo sendo similares, se utilizarmos a estratégia de Bag of Words não veríamos similaridade alguma.

Quando utilizamos embedding para transformar um texto em números, podemos colocar esses números em um gráfico para ver onde ele ficaria posicionado espacialmente.

Observe a imagem abaixo:

<img src="https://miro.medium.com/v2/resize:fit:2000/format:webp/1*SYiW1MUZul1NvL1kc1RxwQ.png">


Note o primeiro exemplo, à esquerda. Note que de "man" para "woman" possui um vetor (quase) idêntico ao vetor de vai de "king" para "queen", porque há uma diferença apenas de gênero de uma palavra para outra, concorda?

O mesmo acontece na imagem à direita. Os vetores são idênticos dos países até suas capitais.

Ou seja, conseguimos uma representação semântica do nosso texto! Tem coisa melhor que isso? Até hoje, não sabemos.

Vamos exemplificar em código:

In [ ]:
# Documentos de exemplo
doc1 = Document(page_content = "Estou com fome", metadata = {})
doc2 = Document(page_content = "Me encontro faminto", metadata = {})

In [ ]:
# Aplicando embedding
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

emb1 = embeddings.embed_query(doc1.page_content)
emb2 = embeddings.embed_query(doc2.page_content)

print(emb1)
print("-="*30)
print(emb2)

[-0.028203917667269707, -0.0031606138218194246, -0.035718388855457306, -0.017829610034823418, -0.017356257885694504, 0.027651673182845116, -0.013648329302668571, 0.04804527387022972, -0.020334433764219284, -0.05214766412973404, 0.007706769742071629, 0.006720618810504675, -0.026744414120912552, 0.016725121065974236, -0.016902627423405647, -0.01830296218395233, 0.03605367988348007, 0.019259529188275337, 0.012888993136584759, 0.019456759095191956, 0.04630964994430542, -0.06023409962654114, -0.0145260039716959, -0.0024924965109676123, 0.036901768296957016, 0.022326458245515823, 0.0037695621140301228, 0.01589675433933735, -0.016054537147283554, -0.03723705932497978, 0.033391073346138, -0.030393173918128014, -0.008352698758244514, -0.0016974123427644372, -0.006454358343034983, 0.02254341170191765, -0.032187968492507935, 0.014082236215472221, 0.000655174080748111, 7.642670243512839e-05, -0.01446683518588543, 0.0107687683776021, 0.0552244558930397, -0.003929811529815197, 0.03078763373196125, 0

In [ ]:
# Calculando a similaridade das duas frases
import numpy as np

similaridade = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
print(f"Similaridade: {similaridade*100:.2f}%")

Similaridade: 73.29%


## Armazenamento
Agora que temos nossos documentos já representados de uma maneira que podemos fazer o cálculo da similaridade, os embeddings, precisamos manter esses embeddings em algum lugar para que possamos realizar a pesquisa.

Precisamos de uma espécie de banco de dados, mas um banco de dados um pouco diferente que chamamos de **banco de dados vetorial (vector database)**.

Diferentemente dos bancos de dados tradicionais, onde realizamos a busca por algum item usando comandos SQL, usamos a busca por similaridade nos bancos de dados vetoriais. **É uma funcionalidade já intrínseca da maioria deles, facilitando o processo para nós**.

Hoje, temos muitos serviços de bancos de dados vetoriais: Pinecone, Qdrant, Chroma... então, qual deles escolher?

Observe abaixo uma tabela que representa uma comparação dos principais bancos de dados vetoriais:

<img src="https://preview.redd.it/my-strategy-for-picking-a-vector-database-a-side-by-side-v0-dw181oiz8esb1.png?width=1870&format=png&auto=webp&s=1858bcae7f68cd9fa32f0d22822f13b9d406c25d">

[Esse site](https://superlinked.com/vector-db-comparison) também é excelente para realizar comparações de vector dbs.

Note que soluções como Pinecone e ElasticSearch não são Open-source, ou seja, de código aberto. Já soluções como Qdrant e Chroma são de código aberto e gratuitos para poder usar.

Vamos utilizar o Milvus para esse exemplo:

In [ ]:
!pip install pymilvus --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.67.1 which is incompatible.


In [ ]:
# Criando o banco de dados vetorial
from pymilvus import MilvusClient

client = MilvusClient("teste_aula.db")

In [ ]:
# Criando uma coleção
if client.has_collection(collection_name = "lean_startup"):
    client.drop_collection(collection_name = "lean_startup")

client.create_collection(
    collection_name = "lean_startup",
    dimension = 1536,
)

In [ ]:
# Carregando os documentos
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PDFPlumberLoader(
    file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
)

docs = loader.load()
docs = [x for x in docs if x.page_content.strip() not in ("\n", "", " ", "\t")]

# Instanciando o splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
docs_splitted = text_splitter.split_documents(docs)

In [ ]:
# Criando e armazenando os embeddings na coleção
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

vectors = embeddings.embed_documents([doc.page_content for doc in docs_splitted])

data = [
    {"id": i, "vector": vectors[i], "text": docs_splitted[i].page_content, "metadata": docs_splitted[i].metadata}
    for i in range(len(vectors))
]

res = client.insert(collection_name="lean_startup", data=data)

In [ ]:
print(res)

{'insert_count': 1348, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [ ]:
# Realizando uma busca semântica
query_vectors = embeddings.embed_query("O que é motor de crescimento?")

res = client.search(
    collection_name = "lean_startup",
    data = [query_vectors],
    filter = "metadata['page'] < 50",
    limit = 2,
    output_fields = ["text", "metadata"],
)

res[0]

[{'id': 92,
  'distance': 0.6697660088539124,
  'entity': {'text': 'um CEO famoso, ele era engenheiro. Em sua oficina, passou dias e noites experimentando a\nmecânica exata para obter o movimento dos cilindros do motor. Cada pequena explosão dentro\ndo cilindro gera força motriz para girar as rodas, e também produz a ignição da próxima\nexplosão. Se o timing desse ciclo de feedback não for gerenciado de modo preciso, o motor\nvai engasgar e deixar de funcionar.\nAs startups possuem um motor semelhante, que denomino motor de crescimento. Os',
   'metadata': {'source': '/content/A Startup Enxuta - Eric Ries.pdf',
    'file_path': '/content/A Startup Enxuta - Eric Ries.pdf',
    'page': 20,
    'total_pages': 210,
    'Author': 'Eric Ries',
    'CreationDate': "D:20150330135456+00'00'",
    'Creator': 'calibre 2.20.0 [http://calibre-ebook.com]',
    'Producer': 'calibre 2.20.0 [http://calibre-ebook.com]',
    'Title': 'A Startup Enxuta'}}},
 {'id': 100,
  'distance': 0.5129328966140747,
 

# Recuperação
<img src="https://i.ibb.co/5dWpf1w/Lead-Entra-na-Lista-11.png">

Até aqui já sabemos tudo desde o processo de ler as informações dos documentos até armazenar esses documentos em um banco de dados vetorial. E eu aposto que você já pensou:
- "Cacete, Anwar! Até aqui já foi coisa demais.""

Exatamente. Uma aplicação de RAG tem diversas variáveis. Diversas! Só na parte de carregamento dos documentos no banco de dados vetorial temos muitas etapas. Mas uma vez que o documento está dentro do banco de dados vetorial, precisamos buscar aqueles que são mais semelhantes à query do usuário, correto? **E é justamente aí que começamos a estudar as metodologias de realizar recuperação, ou *retrieval***.

Ele é o intermediário entre os documentos e a resposta do usuário. Vários bancos de dados vetoriais já implementam sua própria mecânica de recuperação nativamente, onde não precisamos nos preocupar em especificar ou até mesmo escrever algum código que faça isso.

Há, basicamente, três grandes métodos de realizar a recuperação:
- Keyword-matching
- Semantic-matching
- Hybrid Search




## Keyword-Matching (Correspondência de Palavras-Chave)
Esse método é baseado completamente nas palavras-chave que os textos contêm. Também chamamos esse método de correspondência de similaridade lexical.

**Utilizando esses métodos, não olhamos para a semântica do texto nem para o contexto, mas sim para correspondências exatas ou aproximadas de termos presentes no texto**. Temos alguns algoritmos como TF-IDF e BM25 que são muito famosos e utilizados para essa tarefa.

### TF-IDF (Frequência de Termo - Frequência Inversa de Documento)
<img src="https://i.ibb.co/LdhyzTTF/Lead-Entra-na-Lista-12.png">

Esse algoritmo destaca os termos que são frequentes em um chunk específico, mas raros em todos os documentos. **Por exemplo, termos como "de", "para", "é" são meio que desconsiderados, pois aparecem muito nos documentos de maneira geral**. E, claro, são palavras que não precisamos dar tanto foco assim para contabilizar na similaridade.

Podemos utilizar a Bag of Words aqui também! Vale um teste para medirmos qual teria o melhor resultado.

Observe o exemplo abaixo:

In [ ]:
# Realizando a busca
from langchain_community.retrievers import TFIDFRetriever

# Corpus com avaliações de produtos
documentos = [
    "O produto é excelente e chegou rapidamente",
    "Chegou com defeito e atrasado, não recomendo",
    "Atendimento ao cliente foi incrível, produto bom",
    "Entrega rápida, mas o produto estava com defeito"
]

retriever = TFIDFRetriever.from_texts(documentos)

query = "O produto é bom?"

retriever.invoke(query)

[Document(metadata={}, page_content='Atendimento ao cliente foi incrível, produto bom'),
 Document(metadata={}, page_content='O produto é excelente e chegou rapidamente'),
 Document(metadata={}, page_content='Entrega rápida, mas o produto estava com defeito'),
 Document(metadata={}, page_content='Chegou com defeito e atrasado, não recomendo')]

### BM25 (Melhor Correspondência 25)
<img src="https://i.ibb.co/MxjXhZN1/Lead-Entra-na-Lista-13.png">

Esse método é muito famoso para keyword-matching também. **Ele é um método de ranqueamento probabilístico utilizado para medir a relevância de documentos em relação a uma query**.

Ele também é baseado em freqências, assim como o TF-IDF e o Bag of Words que vimos, mas possui algumas funcionalidades adicionais que o torna mais sofisticado e robusto que o simples TF-IDF. Ele junta a frequência dos termos e o comprimento dos documentos pra calcular um score de relevância.

Observe o exemplo de implementação abaixo:



In [ ]:
!pip install rank_bm25 --quiet

In [ ]:
# Realizando a busca
from langchain_community.retrievers import BM25Retriever

# Corpus com avaliações de produtos
documentos = [
    "O produto é excelente e chegou rapidamente",
    "Chegou com defeito e atrasado, não recomendo",
    "Atendimento ao cliente foi incrível, produto bom",
    "Entrega rápida, mas o produto estava com defeito"
]

retriever = BM25Retriever.from_texts(documentos)

query = "O produto é bom?"

retriever.invoke(query)

[Document(metadata={}, page_content='O produto é excelente e chegou rapidamente'),
 Document(metadata={}, page_content='Atendimento ao cliente foi incrível, produto bom'),
 Document(metadata={}, page_content='Entrega rápida, mas o produto estava com defeito'),
 Document(metadata={}, page_content='Chegou com defeito e atrasado, não recomendo')]

## Semantic-Matching (Correspondência Semântica)
Os métodos anteriores eram totalmente baseados em correspondência de palavras-chave - seja exata ou aproximada - e falhava em capturar o significado, a semântica dos textos.

Por isso, temos métodos que são 100% beaseados em retornar com base nos documentos mais similares semanticamente.

Esses embeddings são obtidos geralmente a partir de um modelo de Deep Learning que foi pré-treinado em um gigantesco volume de dados. O objetivo desse modelo é capturar a relação entre as palavras e identificar o significado delas.

Temos diversos modelos que realizam esse tipo de tarefa:
- Modelos de embedding tradicionais: ***Word2Vec, GloVe***
- Modelos de embedding baseados no Transformer: ***BERT, Sentence-BERT, OpenAI Embeddings***
- Modelos de visão para dados não textuais

### Word2Vec (Palavra para Vetor)
<img src="https://i.ibb.co/qMg1kK92/Lead-Entra-na-Lista-14.png">

Você, analisando a imagem acima, pode ter pensando:
- "PQP!"

Seja por achar muito complexo ou por achar muito foda. No meu caso, quando fui estudar pela primeira vez, foi um mix dos dois, mas pendendo mais para o primeiro caso.

Esse modelo funciona da seguinte forma (estou utilizando uma [explicação dada pelo dono do canal StatQuest no YouTube](https://www.youtube.com/watch?v=viZrOnJclY0), que achei fenomenal a didática). Imagine que você tenha duas frases:
- Troll2 is great!
- Gymkata is great!

Como podemos capturar o significado dessas palavras? Bom, podemos ficar treinando a previsão da próxima palavra repetidamente. O que isso significa? Para a primeira frase, se a palavra de entrada for **Troll2**, o modelo tem que prever **is**, se for **is** o modelo tem que prever **great**. Entende? E indicamos qual é a palavra de entrada utilizando o número 1, como você pode ver à esquerda.

- "Ok, Anwar, mas como ele transforma isso em números?"

Observe os números em vermelho logo após as entradas. Esses são os embeddings! Nosso modelo vai aprendendo qual é a melhor combinação de números que faz com que nós acertemos a previsão uma maior quantidade de vezes.

No caso do exemplo, o modelo errou! A palavra com maior pontuação foi **is** ao invés de **great!**. E é aí que o modelo atualiza os pesos para evitar o erro.

Observe o exemplo abaixo usando Word2Vec:

In [ ]:
!pip install gensim --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.7 MB/s eta 0:00:00


In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Documentos de exemplo
documentos = [
    "O produto é excelente e chegou rapidamente",
    "Chegou com defeito e atrasado, não recomendo",
    "Atendimento ao cliente foi incrível, produto bom",
    "Entrega rápida, mas o produto estava com defeito"
]

# Pré-processamento: tokenização
tokens = [simple_preprocess(doc) for doc in documentos]

# Treinando um modelo Word2Vec com os documentos
model = Word2Vec(sentences=tokens, vector_size=300, window=5, min_count=1, workers=4)

# Função para obter o embedding médio de um documento
def get_embedding(doc):
    words = simple_preprocess(doc)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Gerando embeddings para os documentos
document_embeddings = [get_embedding(doc) for doc in documentos]

# Consulta de exemplo
consulta = "O produto é bom?"

# Gerando embedding da consulta
consulta_embedding = get_embedding(consulta)

# Calculando similaridade coseno entre a consulta e os documentos
similaridades = cosine_similarity([consulta_embedding], document_embeddings).flatten()

# Organizando documentos e scores
resultados = sorted(zip(documentos, similaridades), key=lambda x: x[1], reverse=True)

# Exibindo os resultados
print(f"Consulta: {consulta}")
print("\nDocumentos ordenados por relevância:")
for doc, score in resultados:
  if score > 0.40:
    print(f"- Documento: {doc} | Score: {score:.4f}")

Consulta: O produto é bom?

Documentos ordenados por relevância:
- Documento: Atendimento ao cliente foi incrível, produto bom | Score: 0.5691
- Documento: O produto é excelente e chegou rapidamente | Score: 0.4065


### BERT (Bidirectional Encoder Representations from Transformers)
<img src="https://i.ibb.co/KpRKQkkH/Lead-Entra-na-Lista-15.png">

O BERT não lê uma palavra de cada vez, como modelos antigos, mas considera o contexto completo da frase para entender o que está acontecendo. Conseguimos ver isso no nome dele: **Bidirectional**.

Imagine que você tem a frase:
- "O cachorro **correu rápido** porque estava sendo perseguido."

Se perguntarmos ao BERT o que significa "correu rápido", ele vai entender que está ligado à ideia de "fugir" ou "escapar", porque ele já olhou para as palavras antes e depois.

Agora, deixa eu explicar como ele faz isso sem transformar isso num papo chato.

#### Contexto Importa
O BERT pega a frase inteira e processa tudo ao mesmo tempo, de trás para frente e de frente para trás. Por isso é chamado de bidirecional. Ele quer entender o contexto completo.

#### Treinamento com Palavras Mascaradas (Fill in the Blanks)

Imagine que a frase seja:
- "Eu **[MASCARADO]** sorvete no verão."

O trabalho do BERT é adivinhar a palavra "gosto" usando o resto da frase como pista.

#### Previsão de Sequência (Lógica das Frases)
Outra tarefa do BERT é entender se duas frases fazem sentido juntas. Exemplo:
- **Frase 1**: "Eu adoro praia."
- **Frase 2**: "Sempre levo minha prancha."

Ele aprende que essas frases têm uma relação lógica.

#### Tá, mas como ele aprende números?

É como o Word2Vec que falamos anteriormente, mas mais esperto. Cada palavra é transformada em um vetor (uma lista de números). O BERT ajusta esses números para que palavras com significados parecidos fiquem perto umas das outras no espaço multidimensional.

Exemplo:
- "cachorro" e "gato" vão ter números parecidos porque são ambos animais domésticos.
- "rápido" e "devagar" vão ter números bem diferentes porque são opostos.


Observe o exemplo abaixo de aplicação do BERT:

In [ ]:
!pip install transformers --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Carregar o modelo BERT Multilingual e o tokenizer
modelo_nome = "sentence-transformers/bert-base-nli-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(modelo_nome)
modelo = AutoModel.from_pretrained(modelo_nome)

# Textos de exemplo (base de dados)
documentos = [
    "O produto é excelente e chegou rapidamente.",
    "Entrega atrasada e produto com defeito.",
    "Atendimento incrível, produto perfeito.",
    "Chegou quebrado, péssima experiência.",
]

# Função para gerar embeddings usando o BERT
def gerar_embeddings(textos):
    tokens = tokenizer(textos, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = modelo(**tokens)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Média sobre os tokens
    return embeddings

# Gerar embeddings para os documentos
embeddings_documentos = gerar_embeddings(documentos)

# Query para busca
query = "Produto com problemas na entrega."
embedding_query = gerar_embeddings([query])

# Calcular similaridades entre a query e os documentos
similaridades = cosine_similarity(embedding_query, embeddings_documentos)

# Exibir os resultados ordenados
resultados = sorted(
    zip(similaridades[0], documentos), reverse=True, key=lambda x: x[0]
)

print("Resultados da busca para a query:", query)
for similaridade, doc in resultados:
    print(f"Similaridade: {similaridade:.2f} | Documento: {doc}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Resultados da busca para a query: Produto com problemas na entrega.
Similaridade: 0.93 | Documento: Entrega atrasada e produto com defeito.
Similaridade: 0.85 | Documento: Chegou quebrado, péssima experiência.
Similaridade: 0.81 | Documento: O produto é excelente e chegou rapidamente.
Similaridade: 0.80 | Documento: Atendimento incrível, produto perfeito.


## Hybrid Search (Busca Híbrida)
<img src="https://i.ibb.co/zVzhdZR2/Lead-Entra-na-Lista-16.png">

Esse método é um dos melhores, justamente porque não é 100% baseado na similaridade léxica nem 100% baseado na similaridade semântica. **Ele é uma mescla dos dois métodos, onde podemos controlar o peso de cada um deles**.

Imagine um exemplo onde a query seja:
- "Quais são as melhores receitas brasileiras?"

A parte da busca que utiliza keyword-matching vai procurar os documentos que possuem um maior score e, consequentemente, que possuem as palavras-chave "brasileiras" e "receita".

Já a parte responsável pela busca semântica vai procurar pelos documentos que possuem um contexto como "comidas do Brasil" ou algo relacionado.

Com os documentos retornados pelos dois métodos, vamos combinar os scores de cada um e reordernar os documentos.

Observe o exemplo abaixo onde uso o OpenAIEmbeddings e o BM25:

In [ ]:
# Carregando os documentos
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = PDFPlumberLoader(
    file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
)

docs = loader.load()
docs = [x for x in docs if x.page_content.strip() not in ("\n", "", " ", "\t")]
for k, doc in enumerate(docs, start = 1):
  doc.metadata["id"] = k

# Realizando splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 10)
docs_splitted = text_splitter.split_documents(docs)

In [ ]:
# Semantic-Matching
from pymilvus import MilvusClient
from langchain_openai import OpenAIEmbeddings

client = MilvusClient("teste_aula.db")

if client.has_collection(collection_name = "lean_startup"):
    client.drop_collection(collection_name = "lean_startup")

client.create_collection(
    collection_name = "lean_startup",
    dimension = 1536,
)

embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

vectors = embeddings.embed_documents([doc.page_content for doc in docs_splitted])

data = [
    {"id": i, "vector": vectors[i], "text": docs_splitted[i].page_content, "metadata": docs_splitted[i].metadata}
    for i in range(len(vectors))
]

res = client.insert(collection_name="lean_startup", data=data)

In [ ]:
# Keyword-Matching
from rank_bm25 import BM25Okapi
import numpy as np

tokenized_corpus = [doc.page_content.split(" ") for doc in docs_splitted]

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
# Parâmetros
query = "O que é motor de crescimento?"
k = 5
alpha = 0.5

In [ ]:
# BM25
tokenized_query = query.lower().split(" ")
bm25_scores = bm25.get_scores(tokenized_query)
bm25_top_k_indices = np.argsort(bm25_scores)[-k:][::-1]

# Recuperar os top K documentos BM25
bm25_rankings = {doc_id: rank for rank, doc_id in enumerate(bm25_top_k_indices)}

In [ ]:
# Milvus
query_vectors = embeddings.embed_query(query)

semantic_results = client.search(
    collection_name = "lean_startup",
    data = [query_vectors],
    # filter = "metadata['page'] < 50",
    limit = k,
    output_fields = ["text", "id", "metadata"],
)

# Extrair scores e IDs dos resultados semânticos
semantic_rankings = {
    res['id']: rank for rank, res in enumerate(sorted(semantic_results[0], key=lambda x: x['distance']))
}

In [ ]:
# Função que faz o re-ranking
def calculate_rrf(rankings, k=60):
    scores = {}
    for ranker in rankings:
        for doc_id, rank in ranker.items():
            if doc_id not in scores:
                scores[doc_id] = 0
            scores[doc_id] += 1 / (k + rank)
    return scores

In [ ]:
# Scores unidos
combined_scores = calculate_rrf([bm25_rankings, semantic_rankings])

print(combined_scores)

{957: 0.016666666666666666, 1016: 0.01639344262295082, 534: 0.016129032258064516, 964: 0.03149801587301587, 974: 0.015625, 1027: 0.016666666666666666, 92: 0.01639344262295082, 958: 0.016129032258064516, 803: 0.015873015873015872}


In [ ]:
# Pegando os documentos com as K maiores similaridades
top_k_docs = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:k]
result = [(doc_id, docs_splitted[doc_id] if doc_id < len(docs_splitted) else "Documento semântico", score) for doc_id, score in top_k_docs]

In [ ]:
result

[(964,
  Document(metadata={'source': '/content/A Startup Enxuta - Eric Ries.pdf', 'file_path': '/content/A Startup Enxuta - Eric Ries.pdf', 'page': 151, 'total_pages': 210, 'Author': 'Eric Ries', 'CreationDate': "D:20150330135456+00'00'", 'Creator': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Producer': 'calibre 2.20.0 [http://calibre-ebook.com]', 'Title': 'A Startup Enxuta', 'id': 149}, page_content='motores de crescimento. Cada um é como um motor de combustão, girando repetidas vezes.\nQuanto mais rápido o ciclo é completado, mais rápido a empresa crescerá. Cada motor possui\num conjunto intrínseco de métricas que determinam com que rapidez uma empresa pode\ncrescer ao utilizá-lo.\nOS TRÊS MOTORES DE CRESCIMENTO\nVimos na Parte II como é importante que as startups utilizem o tipo certo de métricas –\nmétricas acionáveis – para avaliar o progresso. No entanto, isso deixa uma grande quantidade'),
  0.03149801587301587),
 (957,
  Document(metadata={'source': '/content/A Startup Enxut

# Geração
<img src="https://i.ibb.co/LzFdJ38x/Lead-Entra-na-Lista-17.png">

Já vimos as principais etapas até agora: **a indexação e a recuperação**. Já temos tudo o que precisamos, que é o banco de dados vetorial com os embeddings dentro e o mecanismo que vai recuperar os embeddings mais similares.

Apesar de ter tudo isso, ainda não geramos uma resposta com uma I.A.. De que vale tudo isso sem gerar a resposta, que é o que queremos? É justamente isso que faz a etapa de geração: **geramos uma resposta com a I.A. utilizando os chunks retornados**.

Apesar de parecer muito simples, vou te fazer um questionamento: **basta apenas jogar os chunks retornados no prompt e pedir para gerar a resposta?**

Vamos trabalhar isso agora.

## Retomando o Projeto do Lean Startup
Vou fazer as etapas de indexação e recuperação para podermos implementar a etapa de geração. Para isso, vou fazer o projeto usando o livro Lean Startup aplicando as técnicas que já vimos.

In [ ]:
# Bibliotecas necessárias
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

## INDEXAÇÃO ##

# Carregando os documentos
loader = PDFPlumberLoader(
    file_path = "/content/A Startup Enxuta - Eric Ries.pdf"
)

docs = loader.load()

# Instanciando o splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 60)
docs_splitted = text_splitter.split_documents(docs)

In [ ]:
# Embedding
from uuid import uuid4
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="eric_ries_lean_startup",
    embedding_function=embeddings,
    persist_directory="/content/chroma_db",
)

uuids = [str(uuid4()) for _ in range(len(docs_splitted))]
vector_store.add_documents(documents=docs_splitted, ids=uuids)

# Transformando o vector store em um objeto "pesquisável"
retriever = vector_store.as_retriever()

<ipython-input-94-5e1636572a26>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [ ]:
## RECUPERAÇÃO ##

documentos = retriever.invoke("O que são motores de crescimento?", k = 3)

# Função que formata os documentos retornados
def formataDocumentos(docs):
  return "\n\n".join([f"Documento {k}:\n{doc.page_content}\nPágina do Livro Lean startup que contém as informações do documento {k}: {doc.metadata.get('page')}" for k, doc in enumerate(docs)])

# Formatando os documentos
documentos_formatados = formataDocumentos(documentos)

print(documentos_formatados)

Documento 0:
motores de crescimento. Cada um é como um motor de combustão, girando repetidas vezes.
Quanto mais rápido o ciclo é completado, mais rápido a empresa crescerá. Cada motor possui
um conjunto intrínseco de métricas que determinam com que rapidez uma empresa pode
crescer ao utilizá-lo.
OS TRÊS MOTORES DE CRESCIMENTO
Vimos na Parte II como é importante que as startups utilizem o tipo certo de métricas –
métricas acionáveis – para avaliar o progresso. No entanto, isso deixa uma grande quantidade
em termos de que números devemos medir. De fato, uma das formas mais onerosas de possível
Página do Livro Lean startup que contém as informações do documento 0: 151

Documento 1:
DE ONDE VEM O CRESCIMENTO?
O motor de crescimento é o mecanismo que as startups utilizam para alcançar o crescimento
sustentável. Utilizo a palavra sustentável para excluir todas as atividades ocasionais que
geram um surto de clientes, mas não têm impacto a longo prazo, tais como anúncios isolados
ou uma proeza

## Modelando o prompt da geração
Agora que temos os documentos - e perceba que os deixei até melhor formatados - vamos fazer o prompt. Vou pegar o prompt template para RAG que o próprio langchain tem em seu hub.

In [ ]:
# Pegando o prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

**Question**: {question}

**Context**: {context}

**Answer**:

In [ ]:
# Fazendo uma pergunta ao documento
llm = ChatOpenAI(model = "gpt-4o", temperature = 0)
prompt = PromptTemplate.from_template("""
Você é um assistente para tarefas de resposta a perguntas.
Use as seguintes partes do contexto recuperado para responder à pergunta.
Se você não souber a resposta, diga apenas que não sabe. Use no máximo três frases e mantenha a resposta concisa.
Sempre mencione onde o usuário consegue encontrar as informações (qual página do livro está).
Responda usando espaçamento vertical com \\n

Pergunta:
{question}

Contexto:
{context}

Resposta:
""")

print(prompt.format(question = "O que são motores de crescimento?", context = documentos_formatados))


Você é um assistente para tarefas de resposta a perguntas.
Use as seguintes partes do contexto recuperado para responder à pergunta.
Se você não souber a resposta, diga apenas que não sabe. Use no máximo três frases e mantenha a resposta concisa.
Sempre mencione onde o usuário consegue encontrar as informações (qual página do livro está).
Responda usando espaçamento vertical com \n

Pergunta:
O que são motores de crescimento?

Contexto:
Documento 0:
motores de crescimento. Cada um é como um motor de combustão, girando repetidas vezes.
Quanto mais rápido o ciclo é completado, mais rápido a empresa crescerá. Cada motor possui
um conjunto intrínseco de métricas que determinam com que rapidez uma empresa pode
crescer ao utilizá-lo.
OS TRÊS MOTORES DE CRESCIMENTO
Vimos na Parte II como é importante que as startups utilizem o tipo certo de métricas –
métricas acionáveis – para avaliar o progresso. No entanto, isso deixa uma grande quantidade
em termos de que números devemos medir. De fato, 

In [ ]:
# Criando a chain
chain = prompt | llm | StrOutputParser()
resposta = chain.invoke({"question": "O que são motores de crescimento?", "context": documentos_formatados})

print(resposta)

Motores de crescimento são mecanismos que startups utilizam para alcançar crescimento sustentável, caracterizado por novos clientes surgindo das ações dos clientes passados. Eles funcionam como motores de combustão, onde a rapidez do ciclo determina a velocidade de crescimento da empresa. Informações adicionais podem ser encontradas nas páginas 150 e 151 do livro "Lean Startup".


# Técnicas Avançadas de RAG
Agora que já vimos todas as etapas de um projeto R.A.G., **podemos partir para as técnicas avançadas**. E eu já te adianto: já vimos uma dessas técnicas aqui! Lembra do Hybrid Search? Pois é! É considerado uma técnica avançada.

Hoje, vamos ver as seguintes técnicas:

- Recuperação Hierárquica (Hierarchical Index Retrieval)
- Embedding de Documentos e Perguntas Hipotéticas (Hypothetical Questions & HyDE)
- Recuperação por Janela de Sentença (Sentence Window Retrieval)
- Recuepração do Documento Pai (Parent Document Retriever / Auto-merging Retriever)
- Hybrid Search (já vimos!)
- Transformação da Query (Query Transformation)
- Contexto de Chat (Chat Engine)
- Roteamento de Query (Query Routing)
- RAG Agêntico (Agentic RAG)

Para o código, usarei [esse repositório do GitHub](https://github.com/NirDiamant/RAG_Techniques/tree/main) como referência. É excelente e muito rico!

É, meus amigos e amigas... o trem vai ser cabuloso. Vamos começar?

## Recuperação Hierárquica
<img src="https://i.ibb.co/YFrDy4f7/Lead-Entra-na-Lista-18.png">

Quando temos muitos documentos para buscar, precisamos de uma estratégia eficiente, não é mesmo? É aí que entra a recuperação hierárquica: **criamos dois índices diferentes - um com resumos e outro com chunks dos documentos**.

Mas por que fazer isso? Simples: **primeiro buscamos nos resumos para filtrar os documentos relevantes, e depois buscamos apenas dentro desse grupo específico**.

Pense bem: não é muito mais eficiente do que sair vasculhando documento por documento? Afinal, primeiro identificamos rapidamente quais documentos interessam, e só depois mergulhamos nos detalhes.

Vamos ver na prática como implementar isso?

In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

class RecuperacaoHierarquica:
    def __init__(self, documentos, chunk_size=1000, chunk_overlap=200, metadado_filtro: str = "source"):
        """
        Classe para recuperação hierárquica de documentos utilizando embeddings da OpenAI.

        Args:
            documentos: Lista de documentos no formato Document.
            chunk_size: Tamanho de cada chunk ao dividir os documentos.
            chunk_overlap: Sobreposição entre chunks consecutivos.
        """
        self.documentos = documentos
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.metadado_filtro = metadado_filtro
        self.embeddings = OpenAIEmbeddings()
        self.resumo_store = None
        self.detalhe_store = None
        self.processar_documentos()

    def processar_documentos(self):
        """
        Processa os documentos para criar os vetores de resumo e de chunks detalhados.
        """
        # Criar resumos dos documentos
        modelo_resumo = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
        cadeia_resumo = load_summarize_chain(modelo_resumo, chain_type="map_reduce")

        def resumir_documento(doc):
            """ Resume um documento. """
            saida_resumo = cadeia_resumo.invoke([doc])
            resumo = saida_resumo['output_text']
            return Document(
                page_content=resumo,
                metadata={"origem": "resumo", self.metadado_filtro: doc.metadata.get(self.metadado_filtro, "")}
            )

        # Criar resumos
        resumos = [resumir_documento(doc) for doc in self.documentos]

        # Criar chunks detalhados
        separador_texto = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len
        )
        chunks_detalhados = separador_texto.split_documents(self.documentos)

        # Adicionar metadados aos chunks detalhados
        for i, chunk in enumerate(chunks_detalhados):
            chunk.metadata.update({"id_chunk": i, "origem": "detalhado", self.metadado_filtro: chunk.metadata.get(self.metadado_filtro, "")})

        # Criar armazéns vetoriais
        self.resumo_store = FAISS.from_documents(resumos, self.embeddings)
        self.detalhe_store = FAISS.from_documents(chunks_detalhados, self.embeddings)

    def recuperar(self, consulta, k_resumos=3, k_chunks=5):
        """
        Recupera chunks detalhados com base em uma consulta utilizando a estrutura hierárquica.

        Args:
            consulta: A consulta de pesquisa.
            k_resumos: Número de resumos principais a recuperar.
            k_chunks: Número de chunks detalhados a recuperar por resumo.

        Returns:
            Lista de chunks detalhados relevantes.
        """
        if not self.resumo_store or not self.detalhe_store:
            raise ValueError("Os documentos ainda não foram processados.")

        resumos_relevantes = self.resumo_store.similarity_search(consulta, k=k_resumos)
        chunks_relevantes = []
        print("Resumos:")
        print(resumos_relevantes)
        for resumo in resumos_relevantes:
            source = resumo.metadata.get(self.metadado_filtro, "")
            filtro_source = lambda metadata: metadata.get(self.metadado_filtro, "") == source
            chunks = self.detalhe_store.similarity_search(consulta, k=k_chunks, filter=filtro_source)
            chunks_relevantes.extend(chunks)

        return chunks_relevantes

In [ ]:
# Carregando os documentos
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths = ["https://www.cnnbrasil.com.br/tecnologia/nova-ferramenta-do-chatgpt-acaba-com-buscas-online-especialistas-respondem/",
                 "https://www.cnnbrasil.com.br/tecnologia/perguntamos-ao-chatgpt-por-que-ele-e-melhor-que-o-deepseek-veja-a-resposta/",
                 "https://www.cnnbrasil.com.br/economia/macroeconomia/ue-estabelece-diretrizes-sobre-uso-indevido-de-ia/",
                 "https://www.cnnbrasil.com.br/nacional/centro-oeste/go/golpista-se-passa-por-elon-musk-e-idosa-perde-r150-mil-em-emprestimo-em-go/"]
    )

documentos = loader.load()

In [ ]:
# Criando a instância da classe
recuperacao = RecuperacaoHierarquica(documentos, chunk_size=600, chunk_overlap=200, metadado_filtro = "source")

In [ ]:
# Realizando uma busca com a query desejada
consulta = "quem é elon musk?"
chunks_resultantes = recuperacao.recuperar(consulta, k_resumos=1, k_chunks=3)

# Exibindo os resultados
for i, chunk in enumerate(chunks_resultantes):
    print(f"Resultado {i + 1}:")
    print(f"Metadados: {chunk.metadata}")
    print(f"Conteúdo: {chunk.page_content}\n")

Resumos:
[Document(id='ef177bb1-fcdf-45af-aa5f-98e6ea962b2f', metadata={'origem': 'resumo', 'source': 'https://www.cnnbrasil.com.br/nacional/centro-oeste/go/golpista-se-passa-por-elon-musk-e-idosa-perde-r150-mil-em-emprestimo-em-go/'}, page_content='Uma idosa de 69 anos em Goiás foi vítima de um golpe, perdendo mais de R$150 mil ao acreditar que mantinha um relacionamento virtual com Elon Musk. O golpista, que se fez passar pelo bilionário, manipulou emocionalmente a vítima e pediu empréstimos para "abastecer sua aeronave". Investigações revelam uma organização criminosa por trás do golpe, e a polícia recomenda que familiares monitorem as interações financeiras de idosos com dispositivos móveis.')]
Resultado 1:
Metadados: {'source': 'https://www.cnnbrasil.com.br/nacional/centro-oeste/go/golpista-se-passa-por-elon-musk-e-idosa-perde-r150-mil-em-emprestimo-em-go/', 'title': 'Golpista se passa por Elon Musk e idosa perde R$150 mil em empréstimo em GO | CNN Brasil', 'description': 'Vítima 

## Hypothetical Questions & HyDE
<img src="https://i.ibb.co/hFNBkP6V/Lead-Entra-na-Lista-19.png">

Existem duas técnicas poderosas que envolvem a geração de embeddings especiais. Vamos ver cada uma delas:


### Embeddings de Perguntas
A primeira abordagem é fascinante: **pedimos para a I.A. gerar uma pergunta para cada chunk e criamos embeddings dessas perguntas**.

Por que isso é interessante? Porque na hora da busca, **a similaridade semântica entre a query e a pergunta gerada é muito maior** do que seria com o chunk original. Afinal, ambas são perguntas!

In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

class RecuperacaoHyQuestion:
    def __init__(self, documentos, chunk_size=500, chunk_overlap=100, metadado_filtro="source"):
        """
        Classe para recuperação hierárquica utilizando perguntas geradas automaticamente para cada chunk.

        Args:
            documentos: Lista de documentos no formato Document.
            chunk_size: Tamanho de cada chunk ao dividir os documentos.
            chunk_overlap: Sobreposição entre chunks consecutivos.
            metadado_filtro: Chave de metadados para filtragem.
        """
        self.documentos = documentos
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.metadado_filtro = metadado_filtro
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        self.question_prompt = PromptTemplate(
            input_variables=["chunk"],
            template="""Dado o seguinte trecho de texto, gere 3 perguntas que podem ser respondida por esse conteúdo:

            Trecho: "{chunk}"

            As perguntas devem seguir a seguinte estrutura:
            Pergunta1? Pergunta2? Pergunta3?

            As perguntas devem ser mais gerais e sem mencionar trechos específicos.

            Perguntas:""",
        )
        self.question_chain = self.question_prompt | self.llm

        # Processar documentos e criar armazém vetorial
        self.vectorstore = self.processar_documentos()

    def processar_documentos(self):
        """
        Processa os documentos para criar embeddings de perguntas associadas aos chunks.
        """
        separador_texto = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len
        )
        chunks = separador_texto.split_documents(self.documentos)

        # Gerar perguntas para cada chunk e criar documentos com metadados associados
        documentos_perguntas = []
        for i, chunk in enumerate(chunks):
            pergunta_gerada = self.gerar_pergunta(chunk.page_content)
            doc_pergunta = Document(
                page_content=pergunta_gerada,
                metadata={"id_chunk": i, "chunk_content": chunk.page_content, self.metadado_filtro: chunk.metadata.get(self.metadado_filtro, "")}
            )
            documentos_perguntas.append(doc_pergunta)

        return FAISS.from_documents(documentos_perguntas, self.embeddings)

    def gerar_pergunta(self, chunk):
        """
        Gera uma pergunta para um chunk de texto usando um modelo de IA.
        """
        return self.question_chain.invoke({"chunk": chunk}).content

    def recuperar(self, consulta, k=3):
        """
        Recupera os chunks mais relevantes com base em perguntas associadas.

        Args:
            consulta: A consulta de pesquisa.
            k: Número de chunks a recuperar.

        Returns:
            Lista de chunks de conteúdo correspondentes às perguntas recuperadas.
        """
        perguntas_relevantes = self.vectorstore.similarity_search(consulta, k=k)
        chunks_relevantes = [Document(page_content=p.metadata["chunk_content"], metadata=p.metadata) for p in perguntas_relevantes]

        return chunks_relevantes, perguntas_relevantes

In [ ]:
# Criando a instância da classe
recuperacao_hyquestion = RecuperacaoHyQuestion(documentos, chunk_size=600, chunk_overlap=200)

In [ ]:
# Realizando uma busca com a query desejada
consulta = "quem é elon musk?"
chunks_resultantes, perguntas_relevantes = recuperacao_hyquestion.recuperar(consulta, k=3)

# Exibindo as perguntas hipotéticas geradas
print("Perguntas Relevantes Geradas:\n", perguntas_relevantes)

# Exibindo os resultados da busca
print("\nChunks Recuperados:")
for i, chunk in enumerate(chunks_resultantes):
    print(f"Resultado {i + 1}:")
    print(f"Source: {chunk.metadata.get('source')}")
    print(f"Conteúdo: {chunk.page_content}\n")

Perguntas Relevantes Geradas:
 [Document(id='b0357630-3177-4487-a301-39b16e4e5ac9', metadata={'id_chunk': 68, 'chunk_content': 'Equipe CNN Brasil    Newsletters    Colunistas       Sobre a CNN    Política de Privacidade    Termos de Uso    Fale com a CNN    Distribuição do Sinal    Faça parte da Equipe CNN         Ao vivo    Política    WW    Economia    Esportes    Pop    Viagem & Gastronomia                               seg - sex    Apresentação     Ao vivo            AO VIVO: CNN NOVO DIA - 05/02/2025             Switch      A seguir                        Golpista se passa por Elon Musk e idosa perde R$150 mil em empréstimo em GO   Vítima acreditava manter um relacionamento amoroso virtual com o bilionário por', 'source': 'https://www.cnnbrasil.com.br/nacional/centro-oeste/go/golpista-se-passa-por-elon-musk-e-idosa-perde-r150-mil-em-emprestimo-em-go/'}, page_content='Quem foi a vítima do golpe mencionado no trecho? Qual foi o valor que a idosa perdeu no empréstimo? Em que estado o

### HyDE (Hypothetical Document Embedding)
Já o HyDE faz o caminho inverso: **pedimos para a I.A. gerar uma possível resposta para nossa query e usamos o embedding dessa resposta** junto com o embedding da query para melhorar a qualidade da busca.

Legal, não é? Mas qual técnica escolher? Vamos ver os cenários ideais para cada uma?

In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

class RecuperacaoHyDE:
    def __init__(self, documentos, chunk_size=500, chunk_overlap=100, metadado_filtro="source"):
        """
        Classe para recuperação hierárquica utilizando HyDE (Hypothetical Document Embeddings).

        Args:
            documentos: Lista de documentos no formato Document.
            chunk_size: Tamanho de cada chunk ao dividir os documentos.
            chunk_overlap: Sobreposição entre chunks consecutivos.
            metadado_filtro: Chave de metadados para filtragem.
        """
        self.documentos = documentos
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.metadado_filtro = metadado_filtro
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        # Processar documentos e criar armazém vetorial
        self.vectorstore = self.processar_documentos()

        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""Dada a pergunta '{query}', gere um documento hipotético que responda diretamente a essa pergunta utilizando as principais palavras-chave da área relacionada à pergunta.
            O documento deve ser detalhado e aprofundado, com EXATAMENTE {chunk_size} caracteres.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    def processar_documentos(self):
        """
        Processa os documentos para criar os vetores de embeddings.
        """
        separador_texto = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len
        )
        chunks = separador_texto.split_documents(self.documentos)

        # Adicionar metadados aos chunks
        for i, chunk in enumerate(chunks):
            chunk.metadata.update({"id_chunk": i, self.metadado_filtro: chunk.metadata.get(self.metadado_filtro, "")})

        return FAISS.from_documents(chunks, self.embeddings)

    def gerar_documento_hipotetico(self, consulta):
        """
        Gera um documento hipotético baseado na consulta utilizando o modelo de linguagem.
        """
        input_variables = {"query": consulta, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    def recuperar(self, consulta, k=3):
        """
        Recupera chunks mais relevantes utilizando HyDE para reformular a consulta.

        Args:
            consulta: A consulta de pesquisa.
            k: Número de chunks a recuperar.

        Returns:
            Lista de chunks mais semelhantes e o documento hipotético gerado.
        """
        doc_hipotetico = self.gerar_documento_hipotetico(consulta)
        chunks_relevantes = self.vectorstore.similarity_search(doc_hipotetico, k=k)
        return chunks_relevantes, doc_hipotetico

In [ ]:
# Criando a instância da classe
recuperacao_hyde = RecuperacaoHyDE(documentos, chunk_size=600, chunk_overlap=200)

In [ ]:
# Realizando uma busca com a query desejada
consulta = "quem é elon musk?"
chunks_resultantes, doc_hipotetico = recuperacao_hyde.recuperar(consulta, k=3)

# Exibindo o documento hipotético gerado
print("Documento Hipotético Gerado:\n", doc_hipotetico)

# Exibindo os resultados da busca
print("\nChunks Recuperados:")
for i, chunk in enumerate(chunks_resultantes):
    print(f"Resultado {i + 1}:")
    print(f"Source: {chunk.metadata.get('source')}")
    print(f"Conteúdo: {chunk.page_content}\n")

Documento Hipotético Gerado:
 **Quem é Elon Musk?**

Elon Musk é um empresário e inventor sul-africano, conhecido por sua visão futurista e inovações tecnológicas. Nascido em 28 de junho de 1971, Musk co-fundou a Zip2 e a X.com, que se tornou o PayPal. Ele é o CEO da SpaceX, onde desenvolve foguetes reutilizáveis e busca a colonização de Marte. Como CEO e produto arquiteto da Tesla, Musk revolucionou a indústria automotiva com veículos elétricos e soluções de energia sustentável. Além disso, fundou a Neuralink, focada em interface cérebro-máquina, e a The Boring Company, que visa melhorar o transporte urbano. Musk é uma figura polarizadora, admirada e criticada por suas ideias audaciosas.

Chunks Recuperados:
Resultado 1:
Source: https://www.cnnbrasil.com.br/nacional/centro-oeste/go/golpista-se-passa-por-elon-musk-e-idosa-perde-r150-mil-em-emprestimo-em-go/
Conteúdo: estar em relacionamento amoroso com o bilionário Elon Musk • 16/06/2023REUTERS/Gonzalo Fuentes      Compartilhar matéria

## Sentence Window Retriever
<img src="https://i.ibb.co/mC4qz4g7/Lead-Entra-na-Lista-20.png">

Quando queremos encontrar a melhor correspondência entre uma consulta e um documento, precisamos de precisão, certo? Para isso, **embutimos cada sentença separadamente**, permitindo uma busca eficiente baseada na similaridade de cosseno.

Mas tem um detalhe: **se usarmos apenas uma sentença isolada, podemos perder contexto**. A solução? **Expandimos a janela de contexto**, adicionando `k` sentenças antes e depois da sentença recuperada.

Isso faz toda a diferença: primeiro encontramos a sentença mais relevante no índice, depois ampliamos o contexto para que o LLM tenha mais informações ao gerar a resposta. Assim, garantimos que a busca é precisa e que o modelo raciocina com um contexto mais rico.

Vamos ver como isso funciona na prática?

In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

class RecuperacaoPorSentenca:
    def __init__(self, documentos, chunk_size=500, chunk_overlap=100, metadado_filtro="source", contexto_expandido=2):
        """
        Classe para recuperação baseada em embeddings de sentenças individuais.

        Args:
            documentos: Lista de documentos no formato Document.
            metadado_filtro: Chave de metadados para filtragem.
            contexto_expandido: Número de sentenças antes e depois da sentença encontrada a serem incluídas no contexto.
        """
        self.documentos = documentos
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.metadado_filtro = metadado_filtro
        self.contexto_expandido = contexto_expandido
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        # Processar documentos e criar armazém vetorial
        self.vectorstore, self.sentencas_processadas = self.processar_documentos()

    def processar_documentos(self):
        """
        Processa os documentos para criar embeddings de sentenças individuais.
        """
        separador_texto = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len
        )
        sentencas_processadas = []
        documentos_sentencas = []

        for doc in self.documentos:
            sentencas = separador_texto.split_text(doc.page_content)
            for i, sentenca in enumerate(sentencas):
                documento_sentenca = Document(
                    page_content=sentenca,
                    metadata={"id_sentenca": len(sentencas_processadas), "source": doc.metadata.get(self.metadado_filtro, "")}
                )
                sentencas_processadas.append(sentenca)
                documentos_sentencas.append(documento_sentenca)

        return FAISS.from_documents(documentos_sentencas, self.embeddings), sentencas_processadas

    def recuperar(self, consulta, k=3, N=2):
        """
        Recupera sentenças mais relevantes e expande o contexto ao redor da sentença encontrada.

        Args:
            consulta: A consulta de pesquisa.
            k: Número de sentenças a recuperar.
            N: Número de sentenças antes e depois de cada sentença recuperada.

        Returns:
            Lista de sentenças estendidas para contexto mais amplo.
        """
        sentencas_relevantes = self.vectorstore.similarity_search(consulta, k=k)
        contexto_ampliado = []

        for sentenca_recuperada in sentencas_relevantes:
            id_sentenca = sentenca_recuperada.metadata["id_sentenca"]
            inicio = max(0, id_sentenca - N)
            fim = min(len(self.sentencas_processadas), id_sentenca + N + 1)
            contexto = " | ".join(self.sentencas_processadas[inicio:fim])
            contexto_ampliado.append(Document(page_content=contexto, metadata=sentenca_recuperada.metadata))

        return contexto_ampliado

In [ ]:
# Criando a instância da classe
recuperacao_sentenca = RecuperacaoPorSentenca(documentos, chunk_size=600, chunk_overlap=200)

In [ ]:
# Realizando uma busca com a query desejada
consulta = "quem é elon musk?"
chunks_resultantes = recuperacao_sentenca.recuperar(consulta, k=2, N = 2)

# Exibindo os resultados da busca
print("\nChunks Recuperados:")
for i, chunk in enumerate(chunks_resultantes):
    print(f"Resultado {i + 1}:")
    print(f"Source: {chunk.metadata.get('source')}")
    print(f"Conteúdo: {chunk.page_content}\n")


Chunks Recuperados:
Resultado 1:
Source: https://www.cnnbrasil.com.br/nacional/centro-oeste/go/golpista-se-passa-por-elon-musk-e-idosa-perde-r150-mil-em-emprestimo-em-go/
Conteúdo: Equipe CNN Brasil    Newsletters    Colunistas       Sobre a CNN    Política de Privacidade    Termos de Uso    Fale com a CNN    Distribuição do Sinal    Faça parte da Equipe CNN         Ao vivo    Política    WW    Economia    Esportes    Pop    Viagem & Gastronomia                               seg - sex    Apresentação     Ao vivo            AO VIVO: CNN NOVO DIA - 05/02/2025             Switch      A seguir                        Golpista se passa por Elon Musk e idosa perde R$150 mil em empréstimo em GO   Vítima acreditava manter um relacionamento amoroso virtual com o bilionário por | A seguir                        Golpista se passa por Elon Musk e idosa perde R$150 mil em empréstimo em GO   Vítima acreditava manter um relacionamento amoroso virtual com o bilionário por um aplicativo de mensagens   

## Self-Querying RAG
<img src="https://i.ibb.co/My56ZTBz/Lead-Entra-na-Lista-21.png">

Quando fazemos uma busca, queremos que o sistema entenda exatamente o que estamos pedindo, certo? É aí que entra o **self-querying retrieval**, atuando como um **tradutor inteligente** entre a linguagem natural do usuário e a busca estruturada.

Em vez de apenas procurar por termos semelhantes, **o sistema usa um LLM para extrair tanto a intenção semântica quanto filtros de metadados** (como ano, gênero, classificação). Isso permite refinar a pesquisa e obter resultados muito mais precisos.

Pense no seguinte exemplo: se perguntamos _"Quero um filme de ficção científica após 2010 com nota acima de 8"_, o sistema automaticamente identifica três filtros – **gênero: ficção científica, ano: >2010, nota: >8** – e combina esses critérios com a busca semântica.

Ou seja, em vez de simplesmente retornar qualquer filme relacionado à consulta, ele **filtra e prioriza os mais relevantes**. Bem mais eficiente, não acha?

Vamos ver como implementar isso?

In [ ]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [ ]:
from typing import List
from langchain.chains.query_constructor.schema import AttributeInfo

def definir_campo_metadado(nomes_metadados: List[str], descricoes_metadados: List[str], tipos_metadados: list):
    """
    Define os campos de metadados para a recuperação self-querying com base nos nomes fornecidos.
    """
    return [
        AttributeInfo(
            name=nomes_metadados[k],
            description=descricoes_metadados[k],
            type=str(tipos_metadados[k].__name__),
        ) for k in range(len(nomes_metadados))
    ]

nomes_metadados = ["year", "rating"]
descricoes_metadados = ["O ano de publicação do filme", "A avaliação ou nota do filme, entre 0 e 10"]
tipos_metadados = [int, float]

In [ ]:
import os
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class QueryResponse(BaseModel):
    query: str = Field(
        description=(
            "Texto da busca no conteúdo dos documentos. Essa string representa os termos essenciais que serão utilizados para "
            "encontrar documentos relevantes. Exemplo: 'redes neurais', 'história da computação', 'avanços em IA'."
        )
    )
    filtro: List[str] = Field(
        description=(
            "Lista de filtros a serem aplicados na busca. Cada elemento da lista segue o formato: "
            "'campo|operador|valor'. O campo representa o atributo do metadado a ser filtrado, "
            "Os campos aceitos são: " + ", ".join(nomes_metadados) + "."
            "o operador define a condição de filtragem ('eq' para igual, 'gt' para maior que, 'lt' para menor que, etc.), "
            "e o valor representa a condição. Exemplo: "
            "['ano|gt|2020'] para recuperar documentos publicados depois de 2020; "
            "['categoria|eq|tecnologia', 'autor|eq|Alan Turing'] para filtrar por categoria e autor específico."
        )
    )

class RecuperacaoSelfQuerying:
    def __init__(self, documentos, nomes_metadados: List[str], descricoes_metadados: List[str], tipos_metadados: list, chunk_size=500, chunk_overlap=100):
        """
        Classe para recuperação self-querying utilizando filtros baseados em metadados extraídos automaticamente.

        Args:
            documentos: Lista de documentos no formato Document.
            nomes_metadados: Lista de nomes de metadados aceitos para filtragem.
            chunk_size: Tamanho de cada chunk ao dividir os documentos.
            chunk_overlap: Sobreposição entre chunks consecutivos.
        """
        self.documentos = documentos
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.nomes_metadados = nomes_metadados
        self.descricoes_metadados = descricoes_metadados
        self.tipos_metadados = tipos_metadados
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
        self.metadata_field_info = definir_campo_metadado(nomes_metadados, descricoes_metadados, tipos_metadados)
        self.document_contents = "Texto informativo contendo diversas categorias de conhecimento."

        # Processar documentos e criar armazém vetorial
        self.vectorstore = self.processar_documentos()

        # Criar LLM com saída estruturada
        self.structured_llm = self.llm.with_structured_output(QueryResponse)

    def processar_documentos(self):
        """
        Processa os documentos e cria embeddings armazenados no Chroma.
        """
        return Chroma.from_documents(self.documentos, self.embeddings)

    def recuperar(self, consulta, k=3):
        resposta = self.structured_llm.invoke(consulta)
        structured_query = resposta.model_dump()

        # Constrói lista de sub-filtros
        sub_filters = []
        for filtro in structured_query.get("filtro", []):
            partes = filtro.split("|")
            if len(partes) == 3:
                campo, operador, valor = partes
                if valor.replace(".", "", 1).isdigit():
                    valor = float(valor) if "." in valor else int(valor)

                if operador == "eq":
                    sub_filter = {campo: valor}
                else:
                    sub_filter = {campo: {f"${operador}": valor}}

                sub_filters.append(sub_filter)

        # Monta filtro_dict final
        if not sub_filters:
            # Nenhum filtro
            filtro_dict = {}
        elif len(sub_filters) == 1:
            # Só um filtro => passa diretamente
            filtro_dict = sub_filters[0]
        else:
            # Vários filtros => usa $and
            filtro_dict = {"$and": sub_filters}

        return self.vectorstore.search(
            structured_query["query"],
            search_type="similarity",
            k=k,
            filter=filtro_dict
        )

In [ ]:
# Criando a instância da classe
recuperacao_consulta_propria = RecuperacaoSelfQuerying(docs, nomes_metadados, descricoes_metadados, tipos_metadados, chunk_size=600, chunk_overlap=200)

In [ ]:
# Realizando uma busca com a query desejada
consulta = "quais filmes tem uma nota inferior a 8 e lançados depois de 1990 de ficção científica?"
chunks_resultantes = recuperacao_consulta_propria.recuperar(consulta, k=2)

# Exibindo os resultados da busca
print("\nChunks Recuperados:")
for i, chunk in enumerate(chunks_resultantes):
    print(f"Resultado {i + 1}:")
    print(f"Metadata: {chunk.metadata}")
    print(f"Conteúdo: {chunk.page_content}\n")


Chunks Recuperados:
Resultado 1:
Metadata: {'genre': 'science fiction', 'rating': 7.7, 'year': 1993}
Conteúdo: A bunch of scientists bring back dinosaurs and mayhem breaks loose

Resultado 2:
Metadata: {'genre': 'science fiction', 'rating': 7.7, 'year': 1993}
Conteúdo: A bunch of scientists bring back dinosaurs and mayhem breaks loose



## Parent Document Retriever
<img src="https://i.ibb.co/mVxsZJnq/Lead-Entra-na-Lista-22.png">

O **Auto-Merging Retriever** melhora a recuperação ao combinar precisão com contexto. **Primeiro, buscamos chunks pequenos (leaf chunks), depois, se muitos pertencerem ao mesmo chunk-pai, os substituímos pelo bloco maior antes de enviar ao LLM.**

Isso garante buscas detalhadas sem perder o contexto global, já que a indexação ocorre nos menores chunks, mas a resposta final pode usar trechos maiores quando necessário. **O resultado? Melhor equilíbrio entre granularidade e contexto.**

Vamos ver isso na prática!

In [ ]:
import os
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List

class RecuperacaoDocumentoPai:
    def __init__(self, documentos: List[Document], chunk_size_pai=2000, chunk_size_filho=400):
        """
        Classe para recuperação de documentos utilizando a abordagem de Parent Document Retriever.

        Args:
            documentos: Lista de documentos no formato Document.
            chunk_size_pai: Tamanho dos chunks dos documentos-pai.
            chunk_size_filho: Tamanho dos chunks dos documentos-filho.
        """
        self.documentos = documentos
        self.chunk_size_pai = chunk_size_pai
        self.chunk_size_filho = chunk_size_filho
        self.embeddings = OpenAIEmbeddings()

        # Criar splitters para documentos-pai e documentos-filho
        self.parent_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size_pai)
        self.child_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size_filho)

        # Criar armazenamento vetorial e armazenamento dos documentos
        self.vectorstore = Chroma(collection_name="documentos_pai", embedding_function=self.embeddings)
        self.store = InMemoryStore()

        # Criar o retriever
        self.retriever = ParentDocumentRetriever(
            vectorstore=self.vectorstore,
            docstore=self.store,
            child_splitter=self.child_splitter,
            parent_splitter=self.parent_splitter,
        )

        # Adicionar documentos ao retriever
        self.retriever.add_documents(self.documentos)

    def recuperar(self, consulta: str, k=3):
        """
        Recupera documentos relevantes utilizando a abordagem Parent Document Retriever.

        Args:
            consulta: A consulta de pesquisa em linguagem natural.
            k: Número de documentos a recuperar.

        Returns:
            Lista de documentos relevantes recuperados.
        """
        return self.retriever.invoke(consulta)


In [ ]:
# Importando a classe implementada
recuperacao = RecuperacaoDocumentoPai(documentos, chunk_size_pai = 2000, chunk_size_filho = 400)

In [ ]:
# Definindo a consulta
consulta = "Quem é elon musk?"

# Recuperando documentos mais relevantes
resultados = recuperacao.recuperar(consulta, k=2)

# Exibindo os resultados
print("\n### Documentos Recuperados ###")
for i, doc in enumerate(resultados):
    print(f"\n📄 Documento {i+1}:")
    print(f"{doc.page_content}")



### Documentos Recuperados ###

📄 Documento 1:
Golpista se passa por Elon Musk e idosa perde R$150 mil em empréstimo em GO | CNN Brasil                                           COP 30    Política    Política    Notícias     William Waack      Internacional    Nacional    Economia    Economia    Notícias     Investimentos     Mercado     Cotações     Loterias    Loterias   Mega Sena   Quina   Lotofacil   Lotomania   Duplasena   Loteria Federal   Timemania   Loteca   Dia de Sorte   Super Sete       CNN Money    Entretenimento    Saúde    Esportes    Esportes    Notícias     Olimpíadas     Futebol    Futebol   Brasileirão      Basquete     Automobilismo     Tênis     E-Sports      Tecnologia    Lifestyle    Viagem & Gastronomia    Auto    Educação    CNN Talks    Fórum CNN    Blogs CNN    Colunas CNN       Programação    Equipe CNN Brasil    Newsletters    Colunistas       Sobre a CNN    Política de Privacidade    Termos de Uso    Fale com a CNN    Distribuição do Sinal    Faça parte da

## Query Transformation
<img src="https://i.ibb.co/sdPGzFq1/Lead-Entra-na-Lista-23.png">

A **transformação de queries** melhora a busca de três formas:  
1. **Decomposição** – divide perguntas complexas em sub-queries (ex: _"Compare A e B"_ → _"Características de A"_ + _"Características de B"_).  
2. **Generalização (step-back)** – amplia o contexto antes da busca (ex: _"Impacto ambiental de carros elétricos"_ → _"Energia limpa" + "Produção de baterias"_).  
3. **Reescrita** – reformula a query para mais precisão.  

Muitas vezes, essas estratégias são combinadas para refinar resultados e contextualizar melhor as respostas.

Vamos ver isso na prática!

In [ ]:
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from typing import List

class RecuperacaoTransformacaoDeQuery:
    def __init__(self, documentos: List[Document], estrategia: str, chunk_size=500, chunk_overlap=100):
        """
        Classe para transformação de consultas antes da recuperação de informações.

        Args:
            estrategia: Tipo de transformação a ser aplicada. Pode ser "rewriting", "step-back" ou "sub-query".
            documentos: Lista de documentos a serem processados e armazenados no banco vetorial.
            chunk_size: Tamanho dos chunks ao dividir as consultas.
            chunk_overlap: Sobreposição entre chunks consecutivos.
        """
        if estrategia not in ["rewriting", "step-back", "sub-query"]:
            raise ValueError("A estratégia deve ser 'rewriting', 'step-back' ou 'sub-query'")

        self.estrategia = estrategia
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.documentos = documentos
        self.embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma(collection_name="transformacao_queries", embedding_function=self.embeddings)
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

        # Criar o splitter para dividir os documentos e consultas transformadas
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )

        # Definir prompts conforme a estratégia
        if self.estrategia == "rewriting":
            self.prompt_template = PromptTemplate(
                input_variables=["original_query"],
                template=(
                    "You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system.\n"
                    "Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.\n\n"
                    "Original query: {original_query}\n\n"
                    "Rewritten query:"
                ),
            )
        elif self.estrategia == "step-back":
            self.prompt_template = PromptTemplate(
                input_variables=["original_query"],
                template=(
                    "You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.\n"
                    "Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.\n\n"
                    "Original query: {original_query}\n\n"
                    "Step-back query:"
                ),
            )
        elif self.estrategia == "sub-query":
            self.prompt_template = PromptTemplate(
                input_variables=["original_query"],
                template=(
                    "You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.\n"
                    "Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.\n\n"
                    "Original query: {original_query}\n\n"
                    "Example: What are the impacts of climate change on the environment?\n\n"
                    "Sub-queries:\n"
                    "1. What are the impacts of climate change on biodiversity?\n"
                    "2. How does climate change affect the oceans?\n"
                    "3. What are the effects of climate change on agriculture?\n"
                    "4. What are the impacts of climate change on human health?"
                ),
            )

        self.llm_chain = self.prompt_template | self.llm
        self.processar_documentos()

    def processar_documentos(self):
        """
        Processa os documentos fornecidos e armazena no banco vetorial.
        """
        chunks = self.text_splitter.split_documents(self.documentos)
        self.vectorstore.add_documents(chunks)

    def recuperar(self, consulta: str, k=3):
        """
        Recupera documentos relevantes utilizando a consulta transformada.

        Args:
            consulta: A consulta de pesquisa em linguagem natural.
            k: Número de documentos a recuperar.

        Returns:
            Lista de documentos relevantes recuperados.
        """
        consulta_transformada = self.transformar_consulta(consulta)
        return self.vectorstore.search(consulta_transformada, search_type="similarity", k=k)

    def transformar_consulta(self, consulta: str):
        """
        Aplica a transformação de consulta com base na estratégia definida.

        Args:
            consulta: A consulta original do usuário.

        Returns:
            str | List[str]: A consulta transformada ou uma lista de sub-consultas.
        """
        resposta = self.llm_chain.invoke({"original_query": consulta}).content

        if self.estrategia == "sub-query":
            sub_queries = [q.strip() for q in resposta.split("\n") if q.strip() and not q.strip().startswith("Sub-queries:")]
            return " | ".join([self.text_splitter.split_text(query) for query in sub_queries])

        return resposta

In [ ]:
# Criando uma instância usando a estratégia "rewriting"
recuperacao = RecuperacaoTransformacaoDeQuery(documentos, estrategia="rewriting", chunk_size=600, chunk_overlap=200)

In [ ]:
consulta_original = "Quem é elon musk?"
consulta_transformada = recuperacao.transformar_consulta(consulta_original)

print("🔎 Consulta Transformada:", consulta_transformada)

🔎 Consulta Transformada: Quais são as principais realizações e contribuições de Elon Musk no setor de tecnologia e negócios?


In [ ]:
# Busca documentos relevantes com base na consulta transformada
resultados = recuperacao.recuperar("Quem é elon musk?", k=3)

# Exibir os documentos recuperados
for i, doc in enumerate(resultados):
    print(f"\n📄 Documento {i+1}:")
    print(doc.page_content)


📄 Documento 1:
estar em relacionamento amoroso com o bilionário Elon Musk • 16/06/2023REUTERS/Gonzalo Fuentes      Compartilhar matéria                                    Copiar Link    Uma idosa de 69 anos acreditava manter um relacionamento amoroso virtual com o bilionário Elon Musk. De acordo com as informações prestadas pelos familiares à Polícia Civil do Estado de Goiás, a vítima chegou a realizar dois empréstimos, um deles avaliado em R$ 62 mil e o outro em R$ 92 mil.Segundo a Polícia Civil, o golpista solicitava valores para abastecer a sua suposta aeronave. Houve ainda um pacto de confiança

📄 Documento 2:
estar em relacionamento amoroso com o bilionário Elon Musk • 16/06/2023REUTERS/Gonzalo Fuentes      Compartilhar matéria                                    Copiar Link    Uma idosa de 69 anos acreditava manter um relacionamento amoroso virtual com o bilionário Elon Musk. De acordo com as informações prestadas pelos familiares à Polícia Civil do Estado de Goiás, a vítima cheg

## Chat Engine
<img src="https://i.ibb.co/LzS6LQf2/Query-Transformation-img-src-httpsi-ibb-cosd-PGz-Fq1-Lead-Entra-na-Lista-23-png-A-transformac-a-o-de.png">

Um **Chat Engine** eficiente em RAG mantém o contexto do diálogo via **query compression**, permitindo acompanhamento e referências ao histórico.  

### Abordagens principais:  
1. **ContextChatEngine** – busca contexto e envia ao LLM junto com o histórico.  
2. **CondensePlusContextMode** – condensa histórico + última mensagem, recupera contexto e envia ao LLM.  

Vamos ver isso na prática!

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import Document, HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from typing import List

class RecuperacaoChatEngine:
    def __init__(self, documentos: List[Document], historico: List, engine_type: str, chunk_size=500, chunk_overlap=100):
        """
        Classe para recuperação de contexto em interações de chat usando query compression.

        Args:
            documentos: Lista de documentos a serem processados.
            engine_type: Tipo de engine, pode ser "context" ou "condense".
            chunk_size: Tamanho dos chunks ao dividir os documentos.
            chunk_overlap: Sobreposição entre chunks consecutivos.
            historico: Lista contendo mensagens anteriores da conversa (HumanMessage e AIMessage).
        """
        if engine_type not in ["context", "condense"]:
            raise ValueError("O engine_type deve ser 'context' ou 'condense'")

        self.documentos = documentos
        self.engine_type = engine_type
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.historico = historico
        self.embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma(collection_name="chat_engine", embedding_function=self.embeddings)
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

        # Criar o splitter para dividir os documentos
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )

        self.query_rewrite_prompt = PromptTemplate(
            input_variables=["historico", "consulta"],
            template=(
                "Você é um assistente especializado em reformular consultas para melhorar a recuperação de informações em um sistema RAG.\n"
                "Com base no histórico da conversa fornecido e na nova consulta do usuário, reformule a consulta de forma clara e detalhada, garantindo que referências a contextos anteriores sejam explícitas.\n\n"
                "Histórico da conversa:\n"
                "{historico}\n\n"
                "Consulta do usuário: {consulta}\n\n"
                "Consulta reformulada:"
            )
        )

        self.processar_documentos()

    def processar_documentos(self):
        """
        Processa os documentos e armazena no banco vetorial.
        """
        chunks = self.text_splitter.split_documents(self.documentos)
        self.vectorstore.add_documents(chunks)

    def recuperar(self, consulta: str, k=3):
        """
        Recupera contexto relevante para a interação do chat.

        Args:
            consulta: A consulta do usuário.
            k: Número de documentos a recuperar.

        Returns:
            Lista de documentos relevantes recuperados.
        """
        if self.engine_type == "context":
            return self.vectorstore.search(consulta, search_type="similarity", k=k)

        elif self.engine_type == "condense":
            if not self.historico:
                return self.vectorstore.search(consulta, search_type="similarity", k=k)

            contexto_conversacao = "\n".join([msg.content for msg in self.historico])
            consulta_reformulada = self.llm.invoke(
                self.query_rewrite_prompt.format(historico=contexto_conversacao, consulta=consulta)
            ).content

            print("### Consulta Reformulada ###")
            print(consulta_reformulada)

            return self.vectorstore.search(consulta_reformulada, search_type="similarity", k=k)

In [ ]:
from langchain.schema import HumanMessage, AIMessage

# Criando um histórico fictício de conversa
historico = [
    HumanMessage(content="Quem é elon musk?"),
    AIMessage(content="Elon Musk é um bilionário dono de empresas como Tesla, SpaceX, X e criou uma empresa que se tornou futuramente o paypal.")
]

query = "É verdade que já se passaram por ele?"

In [ ]:
# Criando uma instância usando a estratégia "condense"
recuperacao = RecuperacaoChatEngine(documentos, historico = historico, engine_type="condense", chunk_size=200, chunk_overlap=200)

In [ ]:
# Recuperando documentos relevantes
resultados = recuperacao.recuperar(query, k=3)

# Exibindo os documentos recuperados
print("\n### Documentos Recuperados ###")
for i, doc in enumerate(resultados):
    print(f"\n📄 Documento {i+1}:")
    print(doc.page_content)

### Consulta Reformulada ###
É verdade que pessoas já se passaram por Elon Musk, o bilionário dono de empresas como Tesla, SpaceX e X, e criador de uma empresa que se tornou futuramente o PayPal?

### Documentos Recuperados ###

📄 Documento 1:
um relacionamento amoroso virtual com o bilionário Elon Musk. De acordo com as informações prestadas pelos familiares à Polícia Civil do Estado de Goiás, a vítima chegou a realizar dois empréstimos,

📄 Documento 2:
um relacionamento amoroso virtual com o bilionário Elon Musk. De acordo com as informações prestadas pelos familiares à Polícia Civil do Estado de Goiás, a vítima chegou a realizar dois empréstimos,

📄 Documento 3:
do caso, o suposto Elon Musk adicionou a vítima pelo Facebook e posteriormente, passou a trocar mensagens com a idosa pelo aplicativo do Telegram, no qual os dois mantinham contato por horas e horas,


## Query Routing
<img src="https://i.ibb.co/rR3Mt21k/Query-Transformation-img-src-httpsi-ibb-cosd-PGz-Fq1-Lead-Entra-na-Lista-23-png-A-transformac-a-o-de.png">

O **Query Routing** define o próximo passo do sistema RAG com base na query do usuário. As opções incluem **resumir, buscar em um índice de dados ou combinar múltiplas rotas para gerar uma única resposta.**  

Ele também decide **em qual fonte de dados** buscar – seja entre diferentes bancos (vetorial, gráfico, relacional) ou entre índices hierárquicos (ex: resumos vs. chunks de documentos).  

A escolha da rota é feita via **LLM**, que retorna um formato estruturado para direcionar a busca corretamente – seja para um índice, subcadeias ou até outros agentes.

Vamos ver isso na prática!

In [ ]:
from langchain.schema import Document

# Criando documentos fictícios sobre diferentes temas
documentos_curso = [
    Document(page_content="O curso de maquiagem da influenciadora XYZ ensina técnicas profissionais, uso de produtos e tendências de beleza."),
    Document(page_content="As aulas cobrem desde preparação da pele até looks avançados para eventos especiais.")
]

documentos_stories = [
    Document(page_content="Hoje testei um novo iluminador e estou apaixonada! Ele deixa um glow incrível na pele! 💖"),
    Document(page_content="Dica do dia: Sempre hidratar a pele antes da maquiagem para um acabamento impecável! #DicaDeBeleza")
]

documentos_transcricoes = [
    Document(page_content="No módulo 3 do curso, explico como escolher a base ideal para cada tipo de pele."),
    Document(page_content="A aplicação do contorno deve respeitar a estrutura óssea do rosto para um efeito natural.")
]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Literal

class QueryRoutingOutput(BaseModel):
    db_escolhido: Literal["curso", "stories", "transcricoes"] = Field(
        description=(
            "Nome do vector DB mais apropriado para responder à pergunta.\n"
            "- 'curso': Se a consulta do usuário está relacionada ao conteúdo estruturado do curso de maquiagem da influenciadora, algo mais superficial, incluindo módulos, ementa e preços.\n"
            "- 'stories': Se a consulta se refere a postagens espontâneas e dicas rápidas compartilhadas pela influenciadora em seus stories, como recomendações de produtos, rotina de beleza ou opiniões pessoais.\n"
            "- 'transcricoes': Se a consulta envolve informações detalhadas sobre algum tópico, como explicações aprofundadas sobre técnicas, exemplos demonstrados em vídeo e interações com alunos durante as sessões."
        )
    )

class RecuperacaoRouting:
    def __init__(self, vector_dbs: List[List[Document]], chunk_size=500, chunk_overlap=100):
        """
        Classe para roteamento de consultas entre diferentes bancos vetoriais.

        Args:
            vector_dbs: Lista contendo três listas de documentos, cada uma correspondendo a um vector DB diferente.
            chunk_size: Tamanho dos chunks ao dividir os documentos.
            chunk_overlap: Sobreposição entre chunks consecutivos.
        """
        if len(vector_dbs) != 3:
            raise ValueError("É necessário fornecer exatamente três bancos vetoriais de documentos.")

        self.vector_dbs = vector_dbs
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

        # Criando text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )

        # Criando armazéns vetoriais com FAISS
        self.vectorstore_curso = FAISS.from_documents(self.text_splitter.split_documents(vector_dbs[0]), self.embeddings)
        self.vectorstore_stories = FAISS.from_documents(self.text_splitter.split_documents(vector_dbs[1]), self.embeddings)
        self.vectorstore_transcricoes = FAISS.from_documents(self.text_splitter.split_documents(vector_dbs[2]), self.embeddings)

    def escolher_vector_db(self, consulta: str):
        """
        Decide qual banco vetorial deve ser consultado para responder à pergunta do usuário.

        Args:
            consulta: A consulta do usuário.

        Returns:
            Nome do banco vetorial mais apropriado.
        """
        routing_prompt = PromptTemplate(
            input_variables=["consulta"],
            template=(
                "Você é um assistente especializado em roteamento de queries para um sistema RAG.\n"
                "Dada a seguinte consulta, determine para qual banco de dados vetorial ela deve ser enviada:\n\n"
                "- 'curso': Para perguntas sobre o conteúdo do curso de maquiagem da influenciadora.\n"
                "- 'stories': Para perguntas sobre postagens e stories da influenciadora.\n"
                "- 'transcricoes': Para perguntas sobre transcrições de aulas do curso.\n\n"
                "Consulta: {consulta}\n\n"
                "Retorne apenas o nome do banco vetorial mais apropriado: curso, stories ou transcricoes."
            )
        )

        chain = routing_prompt | self.llm.with_structured_output(QueryRoutingOutput)

        resposta = chain.invoke({"consulta": consulta})
        return resposta.db_escolhido

    def recuperar(self, consulta: str, k=3):
        """
        Recupera documentos relevantes do banco vetorial apropriado.

        Args:
            consulta: A consulta do usuário.
            k: Número de documentos a recuperar.

        Returns:
            Lista de documentos relevantes recuperados.
        """
        db_escolhido = self.escolher_vector_db(consulta)

        if db_escolhido == "curso":
            return self.vectorstore_curso.similarity_search(consulta, k=k)
        elif db_escolhido == "stories":
            return self.vectorstore_stories.similarity_search(consulta, k=k)
        elif db_escolhido == "transcricoes":
            return self.vectorstore_transcricoes.similarity_search(consulta, k=k)
        else:
            return []


In [ ]:
# Criando instância do roteador de recuperação
recuperacao_router = RecuperacaoRouting(
    vector_dbs=[documentos_curso, documentos_stories, documentos_transcricoes],
    chunk_size=300,
    chunk_overlap=50
)

In [ ]:
# Exemplo 1: Pergunta sobre o curso
consulta1 = "Quais são os módulos do curso?"
resultado1 = recuperacao_router.recuperar(consulta1, k=2)
print("\n🔎 Pergunta: ", consulta1)
print("📂 Banco Escolhido: Curso")
for doc in resultado1:
    print("📄", doc.page_content)

# Exemplo 2: Pergunta sobre stories da influenciadora
consulta2 = "Qual produto de maquiagem a influenciadora recomendou recentemente?"
resultado2 = recuperacao_router.recuperar(consulta2, k=2)
print("\n🔎 Pergunta: ", consulta2)
print("📂 Banco Escolhido: Stories")
for doc in resultado2:
    print("📄", doc.page_content)

# Exemplo 3: Pergunta sobre transcrições de aulas
consulta3 = "Como aplicar contorno facial"
resultado3 = recuperacao_router.recuperar(consulta3, k=2)
print("\n🔎 Pergunta: ", consulta3)
print("📂 Banco Escolhido: Transcrições")
for doc in resultado3:
    print("📄", doc.page_content)


🔎 Pergunta:  Quais são os módulos do curso?
📂 Banco Escolhido: Curso
📄 As aulas cobrem desde preparação da pele até looks avançados para eventos especiais.
📄 O curso de maquiagem da influenciadora XYZ ensina técnicas profissionais, uso de produtos e tendências de beleza.

🔎 Pergunta:  Qual produto de maquiagem a influenciadora recomendou recentemente?
📂 Banco Escolhido: Stories
📄 Hoje testei um novo iluminador e estou apaixonada! Ele deixa um glow incrível na pele! 💖
📄 Dica do dia: Sempre hidratar a pele antes da maquiagem para um acabamento impecável! #DicaDeBeleza

🔎 Pergunta:  Como aplicar contorno facial
📂 Banco Escolhido: Transcrições
📄 A aplicação do contorno deve respeitar a estrutura óssea do rosto para um efeito natural.
📄 No módulo 3 do curso, explico como escolher a base ideal para cada tipo de pele.


## Agentic RAG
<img src="https://i.ibb.co/pvgvvDCh/Lead-Entra-na-Lista-27.png">

### **Agents em RAG**  

Os **Agents** permitem que um LLM raciocine e utilize ferramentas externas, como APIs, funções de código ou até outros agentes. No contexto de RAG, isso melhora a recuperação e a síntese de respostas em múltiplos documentos.  

#### **Casos principais:**  
1. **OpenAI Assistants** – Integra histórico de chat, armazenamento de conhecimento, upload de documentos e chamadas de função (API).  
2. **Multi-Document Agents** – Cada documento tem um agente próprio, que pode acessar um **índice vetorial** ou um **índice de resumos**, enquanto um **agente principal** gerencia o roteamento das queries e sintetiza a resposta final.  

Essa abordagem permite comparar informações entre documentos e gerar respostas mais ricas, mas pode ser **mais lenta**, devido a múltiplas chamadas ao LLM. Para grandes volumes de dados, é recomendável simplificar a arquitetura para melhorar a escalabilidade.

In [ ]:
!pip install langgraph arxiv duckduckgo-search --quiet
!pip install faiss-cpu pymupdf --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 59.3 MB/s eta 0:00:00


In [ ]:
class AgenticRAG:
    """
    Classe que contém todo o fluxo RAG (Retrieval-Augmented Generation) com um agente capaz de
    usar ferramentas (tool-using) em Português (pt-BR).
    """

    def __init__(self):
        # --------------------------------------------------------------------------------
        # 1) Criação de Documentos a partir de arquivos e divisão em chunks
        # --------------------------------------------------------------------------------
        import os
        from dotenv import load_dotenv

        # Carrega variáveis de ambiente
        load_dotenv('../.env')

        # Leitura dos arquivos de exemplo
        with open("/content/A-Startup-Enxuta-Eric-Ries.txt", "r", encoding="utf-8") as f:
            self.startup_enxuta = f.read().strip()
        with open("/content/2-kevin-houston-how-to-think-like-a-mathematician.txt", "r", encoding="utf-8") as f:
            self.mathematician = f.read().strip()

        # Importa splitter de texto (ou outro splitter se preferir)
        from langchain_text_splitters import CharacterTextSplitter

        # Inicializa o Text Splitter
        text_splitter = CharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

        # Cria documentos (chunks) do arquivo 'startup_enxuta.txt'
        self.startup_enxuta_texts = text_splitter.create_documents([self.startup_enxuta])

        # Cria documentos (chunks) do arquivo 'mathematician.txt'
        self.mathematician_texts = text_splitter.create_documents([self.mathematician])

        # Adiciona metadados para cada chunk
        for i, doc in enumerate(self.startup_enxuta_texts):
            doc.metadata = {
                'filename': 'startup_enxuta.txt',
                'chunk': i + 1
            }
        for i, doc in enumerate(self.mathematician_texts):
            doc.metadata = {
                'filename': 'mathematician.txt',
                'chunk': i + 1
            }

        # --------------------------------------------------------------------------------
        # 2) Criação e popular instância local do ChromaDB com documentos chunkados
        # --------------------------------------------------------------------------------
        import chromadb
        from langchain_openai import OpenAIEmbeddings
        from langchain_chroma import Chroma

        # Cria cliente Chroma
        chroma_client = chromadb.Client()

        # Cria/pega a collection
        self.collection = chroma_client.get_or_create_collection("test_collection")

        # Modelo de Embeddings
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

        # Criação do Vector Store com Chroma
        self.vector_store = Chroma(
            client=chroma_client,
            collection_name="test_collection",
            embedding_function=self.embeddings,
        )

        # Concatena os documentos e faz ingest na base vetorial
        documents = self.startup_enxuta_texts + self.mathematician_texts
        self.vector_store.add_documents(documents)

        # --------------------------------------------------------------------------------
        # 3) Definição das ferramentas (Tools) para o Agente
        # --------------------------------------------------------------------------------
        from typing import List

        def get_file_list() -> List[str]:
            """
            Retorna uma lista de nomes de arquivos disponíveis no banco de dados,
            para que o usuário possa escolher algum para resumo.
            """
            return ['startup_enxuta.txt', 'mathematician.txt']

        def get_file_content_by_name(filename: str) -> str:
            """
            Retorna o conteúdo do arquivo especificado, extraído do Vector Store.
            Caso o arquivo não exista, retorna mensagem de erro.
            """
            valid_files = ['startup_enxuta.txt', 'mathematician.txt']
            if filename not in valid_files:
                return "ERRO: NOME DE ARQUIVO INVÁLIDO. TENTE OUTRO."

            content_list = self.vector_store.get(where={'filename': filename})['documents']
            content = "\n".join(content_list)
            return content

        def default_rag(query: str) -> str:
            """
            Realiza busca de similaridade no banco vetorial (vector_store)
            para encontrar trechos relevantes relacionados à query do usuário.
            Retorna os trechos (chunks) encontrados.
            """
            results = self.vector_store.similarity_search(query, k=3)
            content_list = [f"* {res.page_content} [{res.metadata}]" for res in results]
            content = "\n".join(content_list)
            return content

        self.get_file_list = get_file_list
        self.get_file_content_by_name = get_file_content_by_name
        self.default_rag = default_rag

        # --------------------------------------------------------------------------------
        # 4) Configuração do LLM e binding com Tools
        # --------------------------------------------------------------------------------
        from langchain_openai import ChatOpenAI

        # Inicializa modelo base
        self.llm = ChatOpenAI(model="gpt-4o-mini")

        # Conjunto de ferramentas (é uma lista de referências às funções)
        self.tools = [self.get_file_list, self.get_file_content_by_name, self.default_rag]

        # "Bind" das ferramentas ao modelo
        self.llm_with_tools = self.llm.bind_tools(self.tools)

        # --------------------------------------------------------------------------------
        # 5) Criação da função Reasoner (nó de raciocínio)
        # --------------------------------------------------------------------------------
        from langchain_core.messages import HumanMessage, SystemMessage

        def reasoner(state):
            """
            Função de raciocínio que decide qual ferramenta usar
            com base na query do usuário e nas mensagens anteriores.
            """
            query = state["query"]
            messages = state["messages"]

            # Mensagem de sistema em Português
            sys_msg = (
                "Você é um Assistente de IA que deve responder exclusivamente em Português (pt-BR). "
                "Você pode usar ferramentas para responder dúvidas relacionadas a arquivos disponíveis. "
                "Se o usuário pedir para resumir um arquivo específico, utilize a ferramenta get_file_list() "
                "para verificar se existe tal arquivo e depois get_file_content_by_name() para obter o conteúdo. "
                "Se a pergunta for sobre algo geral, use default_rag(). "
                "Responda da melhor forma possível em Português."
            )

            # Constrói a mensagem do usuário e adiciona às mensagens
            message = HumanMessage(query)
            messages.append(message)

            # Chama o LLM com as mensagens (system + user + histórico)
            result = [self.llm_with_tools.invoke([sys_msg] + messages)]
            return {"messages": result}

        self.reasoner = reasoner

        # --------------------------------------------------------------------------------
        # 6) Criação do LangGraph (fluxo RAG + Agente)
        # --------------------------------------------------------------------------------
        from typing import Annotated, TypedDict
        from langchain_core.messages import AnyMessage
        from langgraph.graph.message import add_messages

        class State(TypedDict):
            query: str
            messages: Annotated[list[AnyMessage], add_messages]

        from langgraph.graph import StateGraph, START, END
        from langgraph.prebuilt import tools_condition, ToolNode

        # Cria o grafo de estado
        workflow = StateGraph(State)

        # Adiciona nós
        workflow.add_node("reasoner", self.reasoner)
        workflow.add_node("tools", ToolNode(self.tools))

        # Adiciona edges
        workflow.add_edge(START, "reasoner")
        workflow.add_conditional_edges("reasoner", tools_condition)
        workflow.add_edge("tools", "reasoner")

        # Compila o grafo
        self.reAct_graph = workflow.compile()

    def executar_workflow(self, query: str, mensagens_anteriores=None):
        """
        Executa o fluxo RAG com base no 'query' do usuário, usando o grafo reAct_graph.
        Retorna a resposta final do LLM em Português.
        """
        if mensagens_anteriores is None:
            mensagens_anteriores = []
        resposta = self.reAct_graph.invoke({"query": query, "messages": mensagens_anteriores})
        # O resultado é um dicionário com 'messages', que é uma lista de AiMessage/HumanMessage
        # Pega a última resposta do LLM
        resultado = ""
        if "messages" in resposta and len(resposta["messages"]) > 0:
            resultado = resposta["messages"][-1].content
        return resultado.strip()

In [ ]:
# Instanciando a classe
rag = AgenticRAG()

In [ ]:
resposta = rag.executar_workflow("O que é motor de crescimento de uma empresa?")
print(resposta)

O motor de crescimento de uma empresa refere-se ao mecanismo ou às estratégias que uma startup ou negócio utiliza para alcançar um crescimento sustentável ao longo do tempo. Trata-se de um conjunto coordenado de táticas que vão além de ações pontuais, como campanhas publicitárias, e que têm como objetivo gerar um aumento contínuo na base de clientes e nas receitas.

O crescimento sustentável é caracterizado por atividades que não apenas impulsionam um aumento temporário no número de clientes, mas que também estabelecem uma base sólida para um crescimento a longo prazo. Isso envolve entender o que realmente impulsiona a adoção e a retenção de clientes, bem como implementar melhorias constantes nos produtos ou serviços oferecidos.

Portanto, o motor de crescimento é um conceito fundamental para que as empresas possam escalar suas operações de forma eficiente e eficaz, garantindo não apenas a sobrevivência, mas também o sucesso a longo prazo no mercado.


In [ ]:
resposta = rag.executar_workflow("O que é um axioma?")
print(resposta)

Um axioma é uma proposição ou afirmação que é considerada evidente por si mesma e, portanto, não necessita de demonstração. Axiomas são utilizados como pontos de partida em diversos sistemas lógicos e matemáticos. Eles servem como bases a partir das quais outros teoremas e proposições podem ser derivados. Em outros contextos, um axioma pode ser visto como uma regra ou princípio fundamental que orienta o raciocínio ou a teoria em questão.
